In [1]:
import sagemaker
import boto3

import numpy as np                                # For performing matrix operations and numerical processing
import pandas as pd                               # For manipulating tabular data
from time import gmtime, strftime
import os

region = boto3.Session().region_name
smclient = boto3.Session().client('sagemaker')
from sagemaker import get_execution_role

role = get_execution_role()
print(role)
sess = sagemaker.Session()
bucket = "dynamodbtos3zaki"               # Set a default S3 bucket

arn:aws:iam::778449705843:role/service-role/AmazonSageMakerServiceCatalogProductsUseRole


In [2]:
#reader to read data from DynamoDB
def dynamodb_reader(tablename):
    import boto3
    dtable = boto3.resource('dynamodb')
    table = dtable.Table(tablename)
    data = table.scan()
    return data

In [179]:
import boto3
xdtable = boto3.resource('dynamodb')
xtable = xdtable.Table('Activities-prod')
print(xtable)

dynamodb.Table(name='Activities-prod')


In [3]:
def data_extractor(data):
    reqddata = data['Items']
    return reqddata

In [4]:
def key_extractor(reqddatax):
    length = 0
    for i in range(0,len(reqddatax)):
        lstlength = len(reqddatax[i])
        if lstlength>length:
            keylist= reqddatax[i].keys()
            length=len(reqddatax[i])
        return keylist


In [5]:
#function to convert to csv and add data to csv
def convertor_to_csv(keylist,data,filename):
# importing the csv module
    import csv

# my data rows as dictionary objects

# field names
    fields = keylist

# name of csv file
    filenamex = filename

# writing to csv file
    with open(filenamex, 'w', encoding="utf-8-sig") as csvfile:
        # creating a csv dict writer object
        writer = csv.DictWriter(csvfile, fieldnames = fields)

        # writing headers (field names)
        writer.writeheader()

        writer.writerows(data)
        return('Successful')


In [6]:
# str to dict convertor
# using ast.literal_eval()
def str_to_dict(string):
    import ast

# initializing string
    test_string = string
# printing original string
# convert dictionary string to dictionary
    res = ast.literal_eval(test_string)

# return result
    return res


In [7]:
#create an empty csv
def to_csv(name):
    import pandas as pd
    df = pd.DataFrame
    df.to_csv(name)

In [8]:
def s3_uploader(path,name,file):
    boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(path, name)).upload_file(file)
    return True

In [9]:
#upload that csv to our S3 Bucket
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join('Appointments', 'Appointments.csv')).upload_file('Appointments.csv')

In [10]:
#exporting a table
TN1 = 'Activities-prod'
TN2 = 'Content-prod'
TN4 = 'Users-prod'
data1 = dynamodb_reader(TN1)
data2 = dynamodb_reader(TN2)
data4 = dynamodb_reader(TN4)

In [11]:
TN3 = 'Questionnaires-prod'
data3 = dynamodb_reader(TN3)
Rdata3 = data_extractor(data3)
keylist3 = key_extractor(Rdata3)
file3 = convertor_to_csv(keylist3,Rdata3,'Questionnaires-prod')
s3_uploader('Questionnaire','Questionnaire.csv','Questionnaires-prod')
Rdata4 = data_extractor(data4)
keylist4 = key_extractor(Rdata4)

In [12]:
keylist4

dict_keys(['experience', 'phoneNumber', 'modifiedAt', 'userType', 'lastName', 'degree', 'status', 'createdAt', 'createdBy', 'firstName', 'awards', 'userId', 'profilePicture', 'registrationDetails', 'modifiedBy', 'speciality'])

In [13]:
Rdata1 = data_extractor(data1)
Rdata2 = data_extractor(data2)

In [14]:
keylist1 = key_extractor(Rdata1)
keylist2 = key_extractor(Rdata2)

In [15]:
length =0
for i in range(0,len(Rdata4)):
    lstlength = len(Rdata4[i])
    if lstlength>length:
        keylist4= Rdata4[i].keys()
        length=len(Rdata4[i])
print(keylist4)

dict_keys(['experience', 'consultationFeeDetails', 'phoneNumber', 'cognitoUsername', 'modifiedAt', 'userType', 'lastName', 'degree', 'status', 'createdAt', 'createdBy', 'firstName', 'awards', 'cognitoIdentityId', 'userId', 'profilePicture', 'registrationDetails', 'notificationMetadata', 'modifiedBy', 'speciality'])


In [16]:
keylist4 = list(keylist4)
keylist4.append('email')
keylist4.append('leapFavourites')
keylist4.append('weight')
keylist4.append('dateOfBirth')
keylist4.append('height')
keylist4.append('lastMenstrualDate')
keylist4.append('memberships')

In [17]:
file1 = convertor_to_csv(keylist1,Rdata1,'Activities-prod')
file4 = convertor_to_csv(keylist4,Rdata4,'Users-prod')
s3_uploader('Users','Users.csv','Users-prod')

True

In [18]:
length =0
for i in range(0,len(Rdata2)):
    lstlength = len(Rdata2[i])
    if lstlength>length:
        keylist= Rdata2[i].keys()
        length=len(Rdata2[i])
print(keylist)

dict_keys(['metadata', 'modifiedAt', 'status', 'createdAt', 'subtype', 'createdBy', 'weeks', 'trimesters', 'data', 'category', 'id', 'tags', 'modifiedBy', 'type'])


In [19]:
file2 = convertor_to_csv(keylist,Rdata2,'Content-prod')

In [20]:
s3_uploader('Content','Content.csv','Content-prod')

True

In [21]:
s3_uploader('Activities','Activities.csv','Activities-prod')

True

In [22]:
from decimal import *
act = pd.read_csv("Activities-prod")#reading and cleaning act file and storing it in uact
uact = act.drop(columns = ['modifiedAt','createdAt','createdBy','modifiedBy'])
uact = uact[uact.type == 'LEAP_SCORE']
for i in uact.index: #converting details column to dict for further processing and usage
    if type(uact.details[i]) == str:
        uact.details[i] = eval(uact.details[i])

In [23]:
uact.details

14      {'testId': '6961e41a-c3a1-474a-9e40-2ed2b018d4...
17      {'answers': {'6714c9be-551c-472d-8b0a-1575139e...
45      {'score': {}, 'testId': '6961e41a-c3a1-474a-9e...
48      {'answers': {'6714c9be-551c-472d-8b0a-1575139e...
76      {'answers': {'6714c9be-551c-472d-8b0a-1575139e...
                              ...                        
1905    {'answers': {'166dddf6-8789-4019-af9a-db32221c...
1911    {'testId': '6961e41a-c3a1-474a-9e40-2ed2b018d4...
1932    {'testId': '6961e41a-c3a1-474a-9e40-2ed2b018d4...
1938    {'answers': {'6714c9be-551c-472d-8b0a-1575139e...
1940    {'answers': {'c57be6ff-ebdf-45c2-930d-8dcdfe08...
Name: details, Length: 145, dtype: object

In [24]:
con = pd.read_csv("Content-prod")#reading and cleaning con file and storing it in ucon
ucon = con.drop(columns = ['metadata','modifiedAt','createdAt','createdBy','modifiedBy'])
ucon = ucon[ucon.status == 'ACTIVE']
ucon = ucon[ucon.category == 'LEAP']
for i in ucon.index:#converting the data column to dict for further processing
    ucon.data[i] = str_to_dict(ucon.data[i])

In [25]:
ques = pd.read_csv('Questionnaires-prod') #reading and cleaning quest file and storing it in uques
uques = ques.drop(columns = ['metadata','modifiedAt','createdAt','createdBy','modifiedBy'])
uques = uques[uques.status == 'ACTIVE']
for i in uques.index: #converting details column to dict for further processing and usage
    uques.details[i] = eval(uques.details[i])
uques = uques.set_index('id') #set index to id
uques = uques.drop(columns = ['testId','status','type'])

In [26]:
users = pd.read_csv('Users-prod')
uusers = users[users.userType == 'mothers']
uusers = uusers[uusers.status == 'ACTIVE']
uusers = uusers.drop(columns = ['experience','consultationFeeDetails','modifiedAt','degree','awards','createdAt','createdBy',
                                'profilePicture','registrationDetails','notificationMetadata','modifiedBy','speciality','memberships'])
uusers = uusers.drop(columns = ['phoneNumber','cognitoUsername','cognitoIdentityId','email','status','userType'])

In [27]:
uusers.columns

Index(['lastName', 'firstName', 'userId', 'leapFavourites', 'weight',
       'dateOfBirth', 'height', 'lastMenstrualDate'],
      dtype='object')

In [28]:
uusers

,lastName,firstName,userId,leapFavourites,weight,dateOfBirth,height,lastMenstrualDate
18,Jain,Neha,a01814b7-0f02-4159-b766-2b64a05195a8,[],"{'value': '75', 'units': 'kgs'}",1991-05-29,"{'value': Decimal('64'), 'units': 'inches'}",2020-11-01
23,Sheikh,Saif,95ff66c9-a491-4921-9e59-56d108dc6dc7,[],"{'value': '65.65', 'units': 'kgs'}",2000-05-31,"{'value': Decimal('71'), 'units': 'inches'}",2021-03-28
28,Shenoy,Vinayak,a973c169-82d7-4b05-a1c0-a5c3dbccac51,[],"{'value': '80', 'units': 'kgs'}",1982-01-28,"{'value': Decimal('67'), 'units': 'inches'}",2020-09-10
35,Hdhh,Lack,aa5e974b-451c-4b7f-9097-938eb6704422,[],"{'value': '72', 'units': 'kgs'}",1991-02-27,"{'value': Decimal('62'), 'units': 'inches'}",NaN-NaN-NaN
42,VN,Manasi,3410d897-7813-4b3e-a6a7-685d1fb231e4,[],"{'value': '72.3', 'units': 'kgs'}",2003-12-18,"{'value': Decimal('65'), 'units': 'inches'}",2021-03-17
...,...,...,...,...,...,...,...,...
1994,Test,Firoz-test,7776f9a6-75c4-4847-89d3-a94b74167bd2,[],"{'value': '55.5', 'units': 'kgs'}",2004-06-01,"{'value': Decimal('65'), 'units': 'inches'}",2021-05-22
1998,Jain,Vishal,b3353cc5-3a57-46e8-9733-b38d6f94fb42,[],"{'value': '70.00', 'units': 'kgs'}",2001-08-01,"{'value': Decimal('68'), 'units': 'inches'}",2021-05-26
2006,NaN,Aishwarya,cc480da9-99b5-42e4-aaa0-b43c6fa37291,[],"{'value': '60.00', 'units': 'kgs'}",1997-08-15,"{'value': Decimal('65'), 'units': 'inches'}",2021-02-15
2007,Sheikh,Saif,809af84a-c49b-44c3-9426-ecdcc40210b6,[],"{'value': '100.0', 'units': 'kgs'}",1995-06-08,"{'value': Decimal('72'), 'units': 'inches'}",2021-05-24


In [29]:
ucon

,status,subtype,weeks,trimesters,data,category,id,tags,type
0,ACTIVE,PHYSICAL_FITNESS,[Decimal('4')],[Decimal('1')],{'title': 'WHEN YOU SHOULDN’T EXERCISE AT ALL ...,LEAP,1775d9f3-3de5-452a-bac7-4f4357735576,[],PHYSICAL_FITNESS
1,ACTIVE,PHYSICAL_FITNESS,[Decimal('4')],[Decimal('1')],"{'title': 'TRIMESTER TIPS ', 'featureMedia': {...",LEAP,19488ef8-8135-4770-8379-549a166d8114,"['Pregnancy', 'Yoga', 'Exercise', 'Pilates ', ...",PHYSICAL_FITNESS
2,ACTIVE,PHYSICAL_FITNESS,[Decimal('6')],[Decimal('1')],{'title': 'Warning Signs: Working Out While Pr...,LEAP,28f88e8d-a4f7-423a-9716-8e2cc163cf8d,"['Child Development', 'Pregnancy', 'Exercise',...",PHYSICAL_FITNESS
3,ACTIVE,PHYSICAL_FITNESS,[Decimal('28')],[Decimal('3')],"{'title': 'Trimester Tips', 'featureMedia': {'...",LEAP,2cbd2f63-a1f7-4834-9fc1-6baf48ea92e4,[],PHYSICAL_FITNESS
5,ACTIVE,PHYSICAL_FITNESS,[Decimal('4')],[Decimal('1')],{'title': 'WHEN YOU SHOULDN’T EXERCISE AT ALL ...,LEAP,39aac870-13d0-4d93-8285-c46ee2649520,"['Exercise', 'Pregnancy', 'Anemia', 'Hypertens...",PHYSICAL_FITNESS
...,...,...,...,...,...,...,...,...,...
835,ACTIVE,LIFESTYLE,[Decimal('14')],[Decimal('2')],"{'title': 'FIBRE', 'featureMedia': {'image': '...",LEAP,7b864ed9-c589-4164-84f2-9399f5d48bce,"['Child Development', 'Pregnancy', 'Nutrition'...",LIFESTYLE
836,ACTIVE,LIFESTYLE,[Decimal('7')],[Decimal('1')],"{'title': 'Bananas', 'featureMedia': {'image':...",LEAP,7bf74c49-2b36-4c42-92c5-553db02d9815,"['Pregnancy', 'Child Developments', 'Bananas',...",LIFESTYLE
837,ACTIVE,LIFESTYLE,[Decimal('9')],[Decimal('1')],"{'title': 'Basil', 'featureMedia': {'image': '...",LEAP,81ad847b-a227-4a02-80ca-d3216dbe8a1c,"['Pregnancy', 'Child Development', 'Basil', 'I...",LIFESTYLE
838,ACTIVE,LIFESTYLE,[Decimal('1')],[Decimal('1')],{'title': 'HOW A VEGAN DIET COULD AFFECT YOUR ...,LEAP,86895125-806b-4909-b198-bfaf25943dd7,"['Vegan', 'B12', 'Nutrients', 'Iron', 'Choline...",LIFESTYLE


In [30]:
uques

,category,details
id,,
0344caa0-2d71-48b4-9d4b-6a83066c56cc,PHYSICAL_FITNESS,{'question': 'How much time do you spend stand...
0ba5b4aa-c996-47ef-b5eb-3008482bda5b,EMOTION,"{'question': 'In the last month, how often hav..."
0e10ea24-cad7-477c-b9cd-28c87fc92a00,LIFESTYLE,{'question': 'Does your usual diet consist of ...
166dddf6-8789-4019-af9a-db32221cd661,LIFESTYLE,{'question': 'How much sleep do you get in a n...
1b850a07-f898-4440-ab63-849f3cfe4663,EMOTION,"{'question': 'In the last month, how often hav..."
263c80c5-5a09-4c40-9e3a-7cc4643fa201,PHYSICAL_FITNESS,{'question': 'How much time do you spend sitti...
26776b59-2088-400e-82d4-36472b2cf50e,EMOTION,"{'question': 'In the last month, how often hav..."
36dc6b78-837c-4976-8214-79d5c7e95ef7,LIFESTYLE,"{'question': 'Do you frequently eat Flaxseeds,..."
39d80cee-d3a8-42aa-9fde-0e11db43f3d1,PHYSICAL_FITNESS,{'question': 'Do you feel overly short of brea...


In [31]:
uact

,userId,status,details,id,type
14,a01814b7-0f02-4159-b766-2b64a05195a8,ACTIVE,{'testId': '6961e41a-c3a1-474a-9e40-2ed2b018d4...,75d3aa88-f664-41d3-a78a-d7246b9d83b2,LEAP_SCORE
17,95ff66c9-a491-4921-9e59-56d108dc6dc7,ACTIVE,{'answers': {'6714c9be-551c-472d-8b0a-1575139e...,2616ef0d-681c-4efd-8036-de872a5bab81,LEAP_SCORE
45,3410d897-7813-4b3e-a6a7-685d1fb231e4,ACTIVE,"{'score': {}, 'testId': '6961e41a-c3a1-474a-9e...",d3f225a5-cb8a-47fd-bcea-f0bf3fabbfe3,LEAP_SCORE
48,263b167d-f79d-4539-bcf8-bb41dd477d90,ACTIVE,{'answers': {'6714c9be-551c-472d-8b0a-1575139e...,09459b12-7a3b-4285-8421-6e7c09d806d8,LEAP_SCORE
76,210b6961-7162-48d8-b994-b9425d3d2f55,ACTIVE,{'answers': {'6714c9be-551c-472d-8b0a-1575139e...,3022a4d6-8f12-4270-8a3a-19ca7b2d809e,LEAP_SCORE
...,...,...,...,...,...
1905,9af175e6-735f-42ba-b339-917a7a7ba0a7,ACTIVE,{'answers': {'166dddf6-8789-4019-af9a-db32221c...,e01e1b99-5346-4714-acb4-47a9ab79cc3d,LEAP_SCORE
1911,e2414ffc-ff24-442b-9217-1b5c72bb7d71,ACTIVE,{'testId': '6961e41a-c3a1-474a-9e40-2ed2b018d4...,1f9d27c0-3523-460a-8e2f-08a51a1fe269,LEAP_SCORE
1932,1d273f08-0501-4e82-aaad-d5beb58867b2,ACTIVE,{'testId': '6961e41a-c3a1-474a-9e40-2ed2b018d4...,c9bb8db4-6bae-4d93-9295-d7d343b7484c,LEAP_SCORE
1938,ffde9a43-1abf-46d0-bb3a-3bb349e49ea8,ACTIVE,{'answers': {'6714c9be-551c-472d-8b0a-1575139e...,bf18d364-0c3d-41db-9dd9-b1765e4f91dd,LEAP_SCORE


In [32]:
uques1 = uques.copy() #creating a copy so that the actual data isn't lost
uques1 = uques1.drop('6961e41a-c3a1-474a-9e40-2ed2b018d445') #dropping a single empty value
for i in uques1.index: #storing the actual question line by dropping unnecessary keys 
    uques1.details[i] = uques1.details[i]['question']
uques1.details['6714c9be-551c-472d-8b0a-1575139e0e4b']

'In the last month, how often have you felt that you were on top of things ?'

In [33]:
#temp = uact.details[17]['answers'].copy()
#old_key = '6714c9be-551c-472d-8b0a-1575139e0e4b'
#new_key = 'In the last month, how often have you felt that you were on top of things ?' 
#temp[new_key] = temp.pop(old_key)

In [34]:
uact1 = uact.copy() #using the copy 
dropindex=[]
for i in uact1.index: #considering those indexes which have the answers
    if 'answers' in uact1.details[i].keys():
        uact1.details[i]['answers']=uact1.details[i]['answers']
    else:
        uact1.details = uact1.details.drop(i)
        dropindex.append(i)



In [35]:
uact1 = uact1.dropna()

In [36]:
import operator
for i in uact1.index: #arranging in increasing order of scores
    tempx = uact1.details[int(i)]['answers']
    uact1.details[int(i)]['answers'] = dict(sorted(tempx.items(), key=operator.itemgetter(1)))

In [37]:
for i in uact1.index: #replacing questions
    tempx = uact1.details[int(i)]['answers']
    uact1.details[int(i)]['answers'] =dict((uques1.details[key], value) for (key, value) in tempx.items())

In [38]:
uact1.details[17]

{'answers': {'How much sleep do you get in a night?': '1',
  'Does your usual diet consist of nuts (eg. Almonds, Cashews, Peanuts, etc.) ?': '1',
  'Does your usual diet consist of Rawas (a type of fish), Shrimp or Prawns?': '1',
  'Do you participate in Pilates/ Yoga sessions every week ?': '1',
  'Does your usual diet consist of Eggs ?': '1',
  'Does your daily diet include green leafy vegetables ?': '1',
  'Do you often include dessert/ sweet dishes in your diet ?': '1',
  'Do you participate in some outdoor sports (Badminton, Tennis, Swimming, Running, etc) every week ?': '1',
  'Do you frequently eat Flaxseeds, Chia seeds, Walnuts or Edamame ?': '1',
  'Do you feel overly short of breath, dizzy, nauseous or feel overexerted after exercise ?': '1',
  'Do you participate in Spinning/ Cardio/ Zumba sessions every week ?': '2',
  'Do you exercise at least every alternate day ?': '2',
  'Does your usual diet consist of Meat (eg. Chicken, Mutton, Pork, etc) ?': '2',
  'Does your usual d

In [39]:
uact1

,userId,status,details,id,type
17,95ff66c9-a491-4921-9e59-56d108dc6dc7,ACTIVE,{'answers': {'How much sleep do you get in a n...,2616ef0d-681c-4efd-8036-de872a5bab81,LEAP_SCORE
48,263b167d-f79d-4539-bcf8-bb41dd477d90,ACTIVE,{'answers': {'Do you exercise at least every a...,09459b12-7a3b-4285-8421-6e7c09d806d8,LEAP_SCORE
76,210b6961-7162-48d8-b994-b9425d3d2f55,ACTIVE,"{'answers': {'In the last month, how often hav...",3022a4d6-8f12-4270-8a3a-19ca7b2d809e,LEAP_SCORE
94,44de4bd9-bd05-4b7b-bc82-c7ff66d779a3,ACTIVE,{'answers': {'How much water do you drink in a...,9a21a368-a0ae-42fd-9359-3981cad3f3c6,LEAP_SCORE
103,e5b74707-e438-4eb7-a75e-aa84b5647f75,ACTIVE,{'answers': {'What is your general diet ?': '1...,5abcf36b-03ab-4345-93c4-cb33bc8b4ae7,LEAP_SCORE
...,...,...,...,...,...
1825,b59bed52-deff-42a9-9431-ce1207682897,ACTIVE,{'answers': {'How much sleep do you get in a n...,ad647704-86ff-4ebf-9ec6-af6af2ae0256,LEAP_SCORE
1865,d2b3a8a0-b1e3-4bd1-b0b6-a2067445739e,ACTIVE,{'answers': {'Does your usual diet consist of ...,2f233950-cb0f-4111-b121-10b768dcbc99,LEAP_SCORE
1905,9af175e6-735f-42ba-b339-917a7a7ba0a7,ACTIVE,{'answers': {'How much sleep do you get in a n...,e01e1b99-5346-4714-acb4-47a9ab79cc3d,LEAP_SCORE
1938,ffde9a43-1abf-46d0-bb3a-3bb349e49ea8,ACTIVE,{'answers': {'How often do you smoke / consume...,bf18d364-0c3d-41db-9dd9-b1765e4f91dd,LEAP_SCORE


In [40]:
uact.details[17]

{'answers': {'How much sleep do you get in a night?': '1',
  'Does your usual diet consist of nuts (eg. Almonds, Cashews, Peanuts, etc.) ?': '1',
  'Does your usual diet consist of Rawas (a type of fish), Shrimp or Prawns?': '1',
  'Do you participate in Pilates/ Yoga sessions every week ?': '1',
  'Does your usual diet consist of Eggs ?': '1',
  'Does your daily diet include green leafy vegetables ?': '1',
  'Do you often include dessert/ sweet dishes in your diet ?': '1',
  'Do you participate in some outdoor sports (Badminton, Tennis, Swimming, Running, etc) every week ?': '1',
  'Do you frequently eat Flaxseeds, Chia seeds, Walnuts or Edamame ?': '1',
  'Do you feel overly short of breath, dizzy, nauseous or feel overexerted after exercise ?': '1',
  'Do you participate in Spinning/ Cardio/ Zumba sessions every week ?': '2',
  'Do you exercise at least every alternate day ?': '2',
  'Does your usual diet consist of Meat (eg. Chicken, Mutton, Pork, etc) ?': '2',
  'Does your usual d

In [41]:
uid= uact1.userId.unique()

In [42]:
uxam = uact.copy()

In [43]:
z = uxam.userId
uxam = uxam.groupby(z)

In [44]:
uact.details[48]['score']

{'EMOTION': {'value': Decimal('2.5'), 'unit': 'POINTS'},
 'PHYSICAL_FITNESS': {'value': Decimal('2.2222222222222223'),
  'unit': 'POINTS'},
 'LIFESTYLE': {'value': Decimal('3.6666666666666665'), 'unit': 'POINTS'}}

In [45]:
ucon1 = ucon.copy()

In [46]:
ucon1 = ucon.drop(0)

In [47]:
ucon1

,status,subtype,weeks,trimesters,data,category,id,tags,type
1,ACTIVE,PHYSICAL_FITNESS,[Decimal('4')],[Decimal('1')],"{'title': 'TRIMESTER TIPS ', 'featureMedia': {...",LEAP,19488ef8-8135-4770-8379-549a166d8114,"['Pregnancy', 'Yoga', 'Exercise', 'Pilates ', ...",PHYSICAL_FITNESS
2,ACTIVE,PHYSICAL_FITNESS,[Decimal('6')],[Decimal('1')],{'title': 'Warning Signs: Working Out While Pr...,LEAP,28f88e8d-a4f7-423a-9716-8e2cc163cf8d,"['Child Development', 'Pregnancy', 'Exercise',...",PHYSICAL_FITNESS
3,ACTIVE,PHYSICAL_FITNESS,[Decimal('28')],[Decimal('3')],"{'title': 'Trimester Tips', 'featureMedia': {'...",LEAP,2cbd2f63-a1f7-4834-9fc1-6baf48ea92e4,[],PHYSICAL_FITNESS
5,ACTIVE,PHYSICAL_FITNESS,[Decimal('4')],[Decimal('1')],{'title': 'WHEN YOU SHOULDN’T EXERCISE AT ALL ...,LEAP,39aac870-13d0-4d93-8285-c46ee2649520,"['Exercise', 'Pregnancy', 'Anemia', 'Hypertens...",PHYSICAL_FITNESS
6,ACTIVE,PHYSICAL_FITNESS,[Decimal('28')],[Decimal('3')],"{'title': 'THIRD TRIMESTER TIPS. ', 'featureMe...",LEAP,3aababeb-0bf0-460c-9a1e-80a2e38f7abc,[],PHYSICAL_FITNESS
...,...,...,...,...,...,...,...,...,...
835,ACTIVE,LIFESTYLE,[Decimal('14')],[Decimal('2')],"{'title': 'FIBRE', 'featureMedia': {'image': '...",LEAP,7b864ed9-c589-4164-84f2-9399f5d48bce,"['Child Development', 'Pregnancy', 'Nutrition'...",LIFESTYLE
836,ACTIVE,LIFESTYLE,[Decimal('7')],[Decimal('1')],"{'title': 'Bananas', 'featureMedia': {'image':...",LEAP,7bf74c49-2b36-4c42-92c5-553db02d9815,"['Pregnancy', 'Child Developments', 'Bananas',...",LIFESTYLE
837,ACTIVE,LIFESTYLE,[Decimal('9')],[Decimal('1')],"{'title': 'Basil', 'featureMedia': {'image': '...",LEAP,81ad847b-a227-4a02-80ca-d3216dbe8a1c,"['Pregnancy', 'Child Development', 'Basil', 'I...",LIFESTYLE
838,ACTIVE,LIFESTYLE,[Decimal('1')],[Decimal('1')],{'title': 'HOW A VEGAN DIET COULD AFFECT YOUR ...,LEAP,86895125-806b-4909-b198-bfaf25943dd7,"['Vegan', 'B12', 'Nutrients', 'Iron', 'Choline...",LIFESTYLE


In [48]:
droplist=[]
for i in ucon1.index:
    if ucon1.tags[i] == '[]':
        droplist.append(i)
for i in droplist:
    ucon1 = ucon1.drop(i)

In [49]:
#listing all the unique tags (commented for now maybe required in the future for other purposes)(This code takes the tags
#written manually into taglist)
#import itertools
#import re
#taglist = []
#resq = []
#for i in ucon1.index:
#    temp = ucon1.tags[i]
#    res = re.sub(r'[^\w\s]', '', temp)
#    res = res.split()
#    resq.append(res)
#def unique(input_list):
#    return list(set(input_list))
#resq = list(itertools.chain.from_iterable(resq))
#taglist = unique(resq)

In [50]:
temp = uact.details[48].copy()

In [51]:
uactt=uact1.copy()

In [52]:
#sort the data by it's score in uact
for i in uactt.index:
    temp = uactt.details[i]['answers']
    srt = dict(sorted(temp.items(), key=lambda item: item[1]))
    uactt.details[i]['answers'] = srt

In [53]:
for i in uactt.index:#only answers column
    uactt.details[i] = uactt.details[i]['answers']

In [54]:
uactt.details[17]

{'How much sleep do you get in a night?': '1',
 'Does your usual diet consist of nuts (eg. Almonds, Cashews, Peanuts, etc.) ?': '1',
 'Does your usual diet consist of Rawas (a type of fish), Shrimp or Prawns?': '1',
 'Do you participate in Pilates/ Yoga sessions every week ?': '1',
 'Does your usual diet consist of Eggs ?': '1',
 'Does your daily diet include green leafy vegetables ?': '1',
 'Do you often include dessert/ sweet dishes in your diet ?': '1',
 'Do you participate in some outdoor sports (Badminton, Tennis, Swimming, Running, etc) every week ?': '1',
 'Do you frequently eat Flaxseeds, Chia seeds, Walnuts or Edamame ?': '1',
 'Do you feel overly short of breath, dizzy, nauseous or feel overexerted after exercise ?': '1',
 'Do you participate in Spinning/ Cardio/ Zumba sessions every week ?': '2',
 'Do you exercise at least every alternate day ?': '2',
 'Does your usual diet consist of Meat (eg. Chicken, Mutton, Pork, etc) ?': '2',
 'Does your usual diet consist of dal/pulses

In [55]:
ucon1
print(ucon1.data[1].keys())
print(ucon1.tags[1])

dict_keys(['title', 'featureMedia', 'sections'])
['Pregnancy', 'Yoga', 'Exercise', 'Pilates ', 'Sluggish']


In [56]:
ucon1['values'] = ''

In [57]:
def Convert(lst):
    res_dct = {lst[i]: lst[i + 1] for i in range(0, len(lst), 2)}
    return res_dct
def listToString(s): 
    
    # initialize an empty string
    str1 = " " 
    
    # return string  
    return (str1.join(s))

In [58]:
i=0
m=0
n=0
for i in ucon1.index:
    val = ''
    for p in range(0,len(ucon1.data[i]['sections'])):
        
        ftemp = ucon1.data[i]['sections'][p]
        if 'fields' in ftemp.keys(): #fields finder
            for m in range(0,len(ftemp['fields'])):
            
                if 'value' in ftemp['fields'][m].keys(): #finding value keys data                  
                    tempe = ftemp['fields'][m]['value']
                    if type(tempe) is list:
                        tempe = listToString(tempe)
                        tempe = tempe.replace("\u202f", " ")    
                    tempe = tempe.replace("\u202f", " ")
                    val = val + tempe
                
                if 'values' in ftemp['fields'][m].keys():  #finding value keys data
                    temps = ftemp['fields'][m]['values']
                    if type(temps) is list:
                        temps =listToString(temps)
                        temps = temps.replace("\u202f", " ")
                    
                    temps = temps.replace("\u202f", " ")
                    val = val + temps
                
                
                if 'fields' in ftemp['fields'][m].keys(): #finding fields in ucon1.data[i]['sections'][0]['fields'][1]'fields'
                     for n in range(0,len(ftemp['fields'][m]['fields'])):
                            if 'value' in ftemp['fields'][m]['fields'][n].keys(): 
                                tempes = ftemp['fields'][m]['fields'][n]['value']
                                if type(tempes) is list:
                                    tempes = listToString(tempes) 
                                    tempes = tempes.replace("\u202f", " ")
                               
                                tempes = tempes.replace("\u202f", " ")
                                val = val + tempes
                        
                            if 'values' in ftemp['fields'][m]['fields'][n].keys():
                                tempss =ftemp['fields'][m]['fields'][n]['values']
                                if type(tempss) is list:
                                    tempss = listToString(tempss) 
                                    tempss = tempss.replace("\u202f", " ")
                                
                                tempss = tempss.replace("\u202f", " ")
                                val = val + tempss
    ucon1['values'][i] = val
    
        

        


In [59]:
print(ucon1['values'][839])
print(ucon1['data'][839])

An almond, despite being classified as a nut by most, is actually the seed from the fruit of an almond tree.  Fats & Proteins: A single cup of almonds has approximately 30 grams of protein and 71 grams of fat. Almonds contain a high level of fat. However, more than 50% of those fats good fats, which are good for heart and are essential for organ function. However, they also contain bad fats, so it is important not to overdo it. A handful at a time is a great choice, and will give you a nice boost!  Minerals: it is a rich source of fibre, protein, and a number of essential vitamins and minerals, such as B-vitamins, potassium, magnesium, and zinc.   Calories: Roughly 23 almonds would give 130 calories.  Carbs: There are only 5 grams of carbohydrates in a cup of almonds, which represents only 2% of your suggested daily requirement for carbs.  Almonds come packed with plant sterols, which are compounds that prevent your body from absorbing cholesterol. One-third cup of almonds a day, espec

In [60]:
ucon1['values'][17]

"So you have all these methods of tracking intensity...how hard do you want to work? \nIt's a good idea to work at a variety of intensities: Low Intensity: This is about a Level 3-5 on the Perceived Exertion Scale. You might work at this intensity when you're warming up or if you're doing a longer workout, like a long bike ride, walk or run. This might also be an intensity you work at if you take walks throughout the day. Try this intensity about once a week.  Moderate Intensity: This is about a Level 5-6 on the Perceived Exertion Scale and where most of your workouts will fall. Think of getting on a cardio machine or going for a run and being at that place where you can talk, but only a few words. Try this level about 2-3 times a week.  High Intensity: This is about a Level 8-9 on the Perceived Exertion Scale and a level you can only work at for short periods of time. You might work at this intensity when doing high-intensity interval training. Shoot for one, maybe two times a week wi

In [61]:
!pip install rake_nltk
from rake_nltk import Rake   # ensure this is installed

import nltk
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_latest_p36/bin/python -m pip install --upgrade pip' command.


In [62]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /home/ec2-user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ec2-user/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [63]:
from nltk.corpus import stopwords
import re
import string
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from nltk.corpus import stopwords
stop = stopwords.words('english')
stop_words_ = set(stopwords.words('english'))
wn = WordNetLemmatizer()

def black_txt(token):
    return  token not in stop_words_ and token not in list(string.punctuation)  and len(token)>2   
  
def clean_txt(text):
    clean_text = []
    clean_text2 = []
    text = re.sub("'", "",text)
    text=re.sub("(\\d|\\W)+"," ",text) 
    text = text.replace("nbsp", "")
    clean_text = [ wn.lemmatize(word, pos="v") for word in word_tokenize(text.lower()) if black_txt(word)]
    clean_text2 = [word for word in clean_text if black_txt(word)]
    return " ".join(clean_text2)

In [64]:
ucon1['clean']=''

In [65]:
ucon1['clean'] = ucon1['values'].apply(clean_txt)

In [66]:
print(ucon1['values'][1])
print(ucon1['clean'][1])

You can pretty much continue doing exactly what you were doing before pregnancy (including lifting the same amount of weight).  Now is a smart time to get involved in a prenatal yoga or Pilates class; it's a great habit to carry throughout your pregnancy. Plus, you can also get to be around other pregnant women.  If you feel sluggish, try decreasing either the intensity or the duration of your workout rather than skipping the workout altogether (example: drop your jogging pace by 30 seconds a mile, or cut your bike ride a few miles short). 
pretty much continue exactly pregnancy include lift amount weight smart time get involve prenatal yoga pilates class great habit carry throughout pregnancy plus also get around pregnant women feel sluggish try decrease either intensity duration workout rather skip workout altogether example drop jog pace second mile cut bike ride miles short


In [67]:
#initializing tfidf vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

tfidf_con = tfidf_vectorizer.fit_transform((ucon1['values'])) #fitting and transforming the vector
tfidf_con

<140x6402 sparse matrix of type '<class 'numpy.float64'>'
	with 25970 stored elements in Compressed Sparse Row format>

In [68]:
uact1['userId'][17]

'95ff66c9-a491-4921-9e59-56d108dc6dc7'

In [69]:
uact1['questions']= ''
for i in uact1.index:
    temp = list(uact1['details'][i]['answers'].keys())
    uact1['questions'][i] = listToString(temp)
uactx = uact1.copy()

In [70]:
len(uactx['questions'][17])

2372

In [71]:
u = '95ff66c9-a491-4921-9e59-56d108dc6dc7'
index = np.where(uactx['userId'] == u)[0][0]
user_q = uactx.iloc[[index]]
user_q

,userId,status,details,id,type,questions
17,95ff66c9-a491-4921-9e59-56d108dc6dc7,ACTIVE,{'answers': {'How much sleep do you get in a n...,2616ef0d-681c-4efd-8036-de872a5bab81,LEAP_SCORE,How much sleep do you get in a night? Does you...


In [72]:
uactx

,userId,status,details,id,type,questions
17,95ff66c9-a491-4921-9e59-56d108dc6dc7,ACTIVE,{'answers': {'How much sleep do you get in a n...,2616ef0d-681c-4efd-8036-de872a5bab81,LEAP_SCORE,How much sleep do you get in a night? Does you...
48,263b167d-f79d-4539-bcf8-bb41dd477d90,ACTIVE,{'answers': {'Do you exercise at least every a...,09459b12-7a3b-4285-8421-6e7c09d806d8,LEAP_SCORE,Do you exercise at least every alternate day ?...
76,210b6961-7162-48d8-b994-b9425d3d2f55,ACTIVE,"{'answers': {'In the last month, how often hav...",3022a4d6-8f12-4270-8a3a-19ca7b2d809e,LEAP_SCORE,"In the last month, how often have you been ang..."
94,44de4bd9-bd05-4b7b-bc82-c7ff66d779a3,ACTIVE,{'answers': {'How much water do you drink in a...,9a21a368-a0ae-42fd-9359-3981cad3f3c6,LEAP_SCORE,How much water do you drink in a day?
103,e5b74707-e438-4eb7-a75e-aa84b5647f75,ACTIVE,{'answers': {'What is your general diet ?': '1...,5abcf36b-03ab-4345-93c4-cb33bc8b4ae7,LEAP_SCORE,What is your general diet ? Does your usual di...
...,...,...,...,...,...,...
1825,b59bed52-deff-42a9-9431-ce1207682897,ACTIVE,{'answers': {'How much sleep do you get in a n...,ad647704-86ff-4ebf-9ec6-af6af2ae0256,LEAP_SCORE,How much sleep do you get in a night? Does you...
1865,d2b3a8a0-b1e3-4bd1-b0b6-a2067445739e,ACTIVE,{'answers': {'Does your usual diet consist of ...,2f233950-cb0f-4111-b121-10b768dcbc99,LEAP_SCORE,Does your usual diet consist of Rawas (a type ...
1905,9af175e6-735f-42ba-b339-917a7a7ba0a7,ACTIVE,{'answers': {'How much sleep do you get in a n...,e01e1b99-5346-4714-acb4-47a9ab79cc3d,LEAP_SCORE,How much sleep do you get in a night? How much...
1938,ffde9a43-1abf-46d0-bb3a-3bb349e49ea8,ACTIVE,{'answers': {'How often do you smoke / consume...,bf18d364-0c3d-41db-9dd9-b1765e4f91dd,LEAP_SCORE,How often do you smoke / consume alcohol ? In ...


In [73]:
from sklearn.metrics.pairwise import cosine_similarity
user_tfidf = tfidf_vectorizer.transform(user_q['questions'])
cos_similarity_tfidf = map(lambda x: cosine_similarity(user_tfidf, x),tfidf_con)

In [74]:
output2 = list(cos_similarity_tfidf)

In [75]:
def get_recommendation(top, df, scores):
    recommendation = pd.DataFrame(columns = ['userId', 'Id',  'values', 'score'])
    count = 0
    for i in top:
        recommendation.at[count, 'userId'] = u
        recommendation.at[count, 'id'] = ucon1['id'][i]
        recommendation.at[count, 'values'] = ucon1['values'][i]
        recommendation.at[count, 'score'] =  scores[count]
        count += 1
    return recommendation

In [76]:
# to extract key words from Plot to a list
ucon1['Key_words'] = ''   # initializing a new column
r = Rake()   # use Rake to discard stop words (based on english stopwords from NLTK)

for index, row in ucon1.iterrows():
    r.extract_keywords_from_text(row['values'])   # to extract key words from Plot, default in lower case
    key_words_dict_scores = r.get_word_degrees()    # to get dictionary with key words and their scores
    row['Key_words'] = list(key_words_dict_scores.keys())   # to assign list of key words to new column

ucon1

,status,subtype,weeks,trimesters,data,category,id,tags,type,values,clean,Key_words
1,ACTIVE,PHYSICAL_FITNESS,[Decimal('4')],[Decimal('1')],"{'title': 'TRIMESTER TIPS ', 'featureMedia': {...",LEAP,19488ef8-8135-4770-8379-549a166d8114,"['Pregnancy', 'Yoga', 'Exercise', 'Pilates ', ...",PHYSICAL_FITNESS,You can pretty much continue doing exactly wha...,pretty much continue exactly pregnancy include...,"[exactly, feel, sluggish, drop, carry, through..."
2,ACTIVE,PHYSICAL_FITNESS,[Decimal('6')],[Decimal('1')],{'title': 'Warning Signs: Working Out While Pr...,LEAP,28f88e8d-a4f7-423a-9716-8e2cc163cf8d,"['Child Development', 'Pregnancy', 'Exercise',...",PHYSICAL_FITNESS,Pregnant women who exercise should monitor sig...,pregnant women exercise monitor sign potential...,"[encourage, expectant, moms, gynecologists, wa..."
5,ACTIVE,PHYSICAL_FITNESS,[Decimal('4')],[Decimal('1')],{'title': 'WHEN YOU SHOULDN’T EXERCISE AT ALL ...,LEAP,39aac870-13d0-4d93-8285-c46ee2649520,"['Exercise', 'Pregnancy', 'Anemia', 'Hypertens...",PHYSICAL_FITNESS,Exercising during pregnancy does the average p...,exercise pregnancy average pregnant mom baby g...,"[poorly, controlled, type, 1, diabetes, ask, i..."
13,ACTIVE,PHYSICAL_FITNESS,[Decimal('5')],[Decimal('1')],"{'title': 'BEST EXERCISES DURING PREGNANCY. ',...",LEAP,9ae77239-924a-49c6-8dd8-1cc7128e8210,"['Pregnancy', 'Child Development', 'Pilates', ...",PHYSICAL_FITNESS,If you have access to prenatal exercise classe...,access prenatal exercise class sign workouts m...,"[emphasis, good, time, pregnant, ph, motion, m..."
14,ACTIVE,PHYSICAL_FITNESS,[Decimal('7')],[Decimal('1')],"{'title': 'PRENATAL YOGA', 'featureMedia': {'i...",LEAP,9bcfb27f-5b46-40e9-8cc6-e0a675356bc4,"['Pregnancy', 'Child Development', 'Yoga', 'Ex...",PHYSICAL_FITNESS,Prenatal yoga sounds like a healthy practice t...,prenatal yoga sound like healthy practice adop...,"[relax, fear, deep, prenatal, yoga, healthy, b..."
...,...,...,...,...,...,...,...,...,...,...,...,...
835,ACTIVE,LIFESTYLE,[Decimal('14')],[Decimal('2')],"{'title': 'FIBRE', 'featureMedia': {'image': '...",LEAP,7b864ed9-c589-4164-84f2-9399f5d48bce,"['Child Development', 'Pregnancy', 'Nutrition'...",LIFESTYLE,This is plant material like carbohydrates and ...,plant material like carbohydrates substances r...,"[mostly, consisting, disorders, 1g, per, 100g,..."
836,ACTIVE,LIFESTYLE,[Decimal('7')],[Decimal('1')],"{'title': 'Bananas', 'featureMedia': {'image':...",LEAP,7bf74c49-2b36-4c42-92c5-553db02d9815,"['Pregnancy', 'Child Developments', 'Bananas',...",LIFESTYLE,Bananas are rich in potassium and offer quick ...,bananas rich potassium offer quick energy figh...,"[easily, available, fruit, helps, gas, yoghurt..."
837,ACTIVE,LIFESTYLE,[Decimal('9')],[Decimal('1')],"{'title': 'Basil', 'featureMedia': {'image': '...",LEAP,81ad847b-a227-4a02-80ca-d3216dbe8a1c,"['Pregnancy', 'Child Development', 'Basil', 'I...",LIFESTYLE,"Basil is also packed with iron, vital for keep...",basil also pack iron vital keep energy level c...,"[nutrients, phosphorus, oxidizing, cholesterol..."
838,ACTIVE,LIFESTYLE,[Decimal('1')],[Decimal('1')],{'title': 'HOW A VEGAN DIET COULD AFFECT YOUR ...,LEAP,86895125-806b-4909-b198-bfaf25943dd7,"['Vegan', 'B12', 'Nutrients', 'Iron', 'Choline...",LIFESTYLE,By Zaria Gorvett \n28th January 2020 \nThe veg...,zaria gorvett january vegan diet low case enti...,"[body, cannot, make, enough, acad, age, sunshi..."


In [77]:
for i in ucon1.index:
    ucon1['Key_words'][i] = listToString(ucon1['Key_words'][i])

In [78]:
# to generate the count matrix
count = CountVectorizer()
count_matrix = count.fit_transform(ucon1['Key_words'])
count_matrix

<140x6278 sparse matrix of type '<class 'numpy.int64'>'
	with 21342 stored elements in Compressed Sparse Row format>

In [79]:
# to extract key words from Plot to a list
uact1['Key_words'] = ''   # initializing a new column
r = Rake()   # use Rake to discard stop words (based on english stopwords from NLTK)

for index, row in uact1.iterrows():
    r.extract_keywords_from_text(row['questions'])   # to extract key words from Plot, default in lower case
    key_words_dict_scores = r.get_word_degrees()    # to get dictionary with key words and their scores
    row['Key_words'] = list(key_words_dict_scores.keys())   # to assign list of key words to new column

uact1

,userId,status,details,id,type,questions,Key_words
17,95ff66c9-a491-4921-9e59-56d108dc6dc7,ACTIVE,{'answers': {'How much sleep do you get in a n...,2616ef0d-681c-4efd-8036-de872a5bab81,LEAP_SCORE,How much sleep do you get in a night? Does you...,"[fish, ),, shrimp, felt, nervous, often, inclu..."
48,263b167d-f79d-4539-bcf8-bb41dd477d90,ACTIVE,{'answers': {'Do you exercise at least every a...,09459b12-7a3b-4285-8421-6e7c09d806d8,LEAP_SCORE,Do you exercise at least every alternate day ?...,"[fish, ),, shrimp, felt, nervous, often, inclu..."
76,210b6961-7162-48d8-b994-b9425d3d2f55,ACTIVE,"{'answers': {'In the last month, how often hav...",3022a4d6-8f12-4270-8a3a-19ca7b2d809e,LEAP_SCORE,"In the last month, how often have you been ang...","[fish, ),, shrimp, felt, nervous, often, inclu..."
94,44de4bd9-bd05-4b7b-bc82-c7ff66d779a3,ACTIVE,{'answers': {'How much water do you drink in a...,9a21a368-a0ae-42fd-9359-3981cad3f3c6,LEAP_SCORE,How much water do you drink in a day?,"[day, much, water, drink]"
103,e5b74707-e438-4eb7-a75e-aa84b5647f75,ACTIVE,{'answers': {'What is your general diet ?': '1...,5abcf36b-03ab-4345-93c4-cb33bc8b4ae7,LEAP_SCORE,What is your general diet ? Does your usual di...,"[fish, ),, shrimp, meat, edamame, cashews, fel..."
...,...,...,...,...,...,...,...
1825,b59bed52-deff-42a9-9431-ce1207682897,ACTIVE,{'answers': {'How much sleep do you get in a n...,ad647704-86ff-4ebf-9ec6-af6af2ae0256,LEAP_SCORE,How much sleep do you get in a night? Does you...,"[fish, ),, shrimp, rawas, prawns, type, get, n..."
1865,d2b3a8a0-b1e3-4bd1-b0b6-a2067445739e,ACTIVE,{'answers': {'Does your usual diet consist of ...,2f233950-cb0f-4111-b121-10b768dcbc99,LEAP_SCORE,Does your usual diet consist of Rawas (a type ...,"[fish, ),, shrimp, felt, nervous, often, inclu..."
1905,9af175e6-735f-42ba-b339-917a7a7ba0a7,ACTIVE,{'answers': {'How much sleep do you get in a n...,e01e1b99-5346-4714-acb4-47a9ab79cc3d,LEAP_SCORE,How much sleep do you get in a night? How much...,"[get, drink, night, day, much, water, sleep]"
1938,ffde9a43-1abf-46d0-bb3a-3bb349e49ea8,ACTIVE,{'answers': {'How often do you smoke / consume...,bf18d364-0c3d-41db-9dd9-b1765e4f91dd,LEAP_SCORE,How often do you smoke / consume alcohol ? In ...,"[stressed, felt, nervous, something, personal,..."


In [80]:
for i in uact1.index:
    uact1['Key_words'][i] = listToString(uact1['Key_words'][i])

In [81]:
count = CountVectorizer()
count_matrix2 = count.fit_transform(uact1['Key_words'])
count_matrix2

<87x128 sparse matrix of type '<class 'numpy.int64'>'
	with 8048 stored elements in Compressed Sparse Row format>

In [82]:
print(uact1['questions'][17])
print(uact1['details'][17])

How much sleep do you get in a night? Does your usual diet consist of nuts (eg. Almonds, Cashews, Peanuts, etc.) ? Does your usual diet consist of Rawas (a type of fish), Shrimp or Prawns? Do you participate in Pilates/ Yoga sessions every week ? Does your usual diet consist of Eggs ? Does your daily diet include green leafy vegetables ? Do you often include dessert/ sweet dishes in your diet ? Do you participate in some outdoor sports (Badminton, Tennis, Swimming, Running, etc) every week ? Do you frequently eat Flaxseeds, Chia seeds, Walnuts or Edamame ? Do you feel overly short of breath, dizzy, nauseous or feel overexerted after exercise ? Do you participate in Spinning/ Cardio/ Zumba sessions every week ? Do you exercise at least every alternate day ? Does your usual diet consist of Meat (eg. Chicken, Mutton, Pork, etc) ? Does your usual diet consist of dal/pulses ? Do you have a fruit every day ? In the last month, how often have you felt that you have not been able to control th

In [83]:
corr_df = pd.DataFrame(index = ucon1.id)

In [84]:
for i in range(0,len(uques1)):
    temp = uques1.details[i]
    corr_df[temp] = ''

In [85]:
corr_df

,How much time do you spend standing or walking during the day ?,"In the last month, how often have you found that you could not cope with all the things that you had to do ?","Does your usual diet consist of Rawas (a type of fish), Shrimp or Prawns?",How much sleep do you get in a night?,"In the last month, how often have you felt difficulties were pilling up so high that you could not overcome them ?",How much time do you spend sitting on the computer/ watching TV/ talking on the phone ?,"In the last month, how often have you felt nervous and stressed ?","Do you frequently eat Flaxseeds, Chia seeds, Walnuts or Edamame ?","Do you feel overly short of breath, dizzy, nauseous or feel overexerted after exercise ?",Does your usual diet consist of Eggs ?,...,How often do you smoke / consume alcohol ?,"Does your usual diet consist of Meat (eg. Chicken, Mutton, Pork, etc) ?","Does your usual diet consist of nuts (eg. Almonds, Cashews, Peanuts, etc.) ?",Does your daily diet include green leafy vegetables ?,"Do you participate in some outdoor sports (Badminton, Tennis, Swimming, Running, etc) every week ?","In the last month, how often have you felt that things were going your way?",Do you participate in Pilates/ Yoga sessions every week ?,In the last one month how often have you been upset because of something that happened unexpectedly ?,"Do you do the chores(cleaning, cooking or laundry) at home and if so, for how long ?",Do you often include dessert/ sweet dishes in your diet ?
id,,,,,,,,,,,,,,,,,,,,,
19488ef8-8135-4770-8379-549a166d8114,,,,,,,,,,,...,,,,,,,,,,
28f88e8d-a4f7-423a-9716-8e2cc163cf8d,,,,,,,,,,,...,,,,,,,,,,
39aac870-13d0-4d93-8285-c46ee2649520,,,,,,,,,,,...,,,,,,,,,,
9ae77239-924a-49c6-8dd8-1cc7128e8210,,,,,,,,,,,...,,,,,,,,,,
9bcfb27f-5b46-40e9-8cc6-e0a675356bc4,,,,,,,,,,,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7b864ed9-c589-4164-84f2-9399f5d48bce,,,,,,,,,,,...,,,,,,,,,,
7bf74c49-2b36-4c42-92c5-553db02d9815,,,,,,,,,,,...,,,,,,,,,,
81ad847b-a227-4a02-80ca-d3216dbe8a1c,,,,,,,,,,,...,,,,,,,,,,


In [86]:
ucon2 = ucon1.copy()

In [87]:
ucon2 = ucon2.set_index(ucon2['id'])

In [88]:
import math
import re
from collections import Counter
WORD = re.compile(r"\w+")
def get_cosine(vec1, vec2):
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    sum1 = sum([vec1[x] ** 2 for x in list(vec1.keys())])
    sum2 = sum([vec2[x] ** 2 for x in list(vec2.keys())])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator


def text_to_vector(text):
    words = WORD.findall(text)
    return Counter(words)

In [89]:
for i in corr_df.columns:
    for j in corr_df.index:
        text1 = i
        text2 = ucon2['values'][j]

        vector1 = text_to_vector(text1)
        vector2 = text_to_vector(text2)

        cosine = get_cosine(vector1, vector2)
        corr_df[i][j]=cosine

In [90]:
ucon2['Key_words']['796189df-18ed-44a9-9c3a-d17b15319c64']

'sexual stimulation body preparing mouth engorged sex late let breasts reassure soft sometimes prompt premilk good idea noticing produce particularly bleeding oxytocin — uterus drip may surprise hormone released uncommon panic true women start scientific evidence first time orgasm could possibly speed things along deep penetration cervix bring pregnancy woman ’ little spotting doctor know baby notice generally harmless release normal part colostrum third trimester'

In [91]:
corr_df['Do you have a fruit every day ?'].astype(str).astype(float).nlargest(10)

id
b2f3f52b-a703-49c0-9cdc-eaa399ccc81c    0.337127
09e718a8-ad3e-4b3f-9868-c2eacc315632    0.291950
6fd4e033-3aae-4a27-9a1c-be3345be834b    0.285000
5bc1b7c9-2021-4f90-b9a7-b2cf88faae66    0.280351
6dc81491-43c1-441f-9a0b-5da872c49aab    0.271160
064aa5bc-3d41-41d9-ae38-eea0aeb7a90e    0.241729
19488ef8-8135-4770-8379-549a166d8114    0.231908
892d8c8c-e8a8-4be5-9cdc-de99f156cb57    0.222384
28f88e8d-a4f7-423a-9716-8e2cc163cf8d    0.221610
e746ecb0-2b6b-4cb6-9148-c59ee07630a3    0.219687
Name: Do you have a fruit every day ?, dtype: float64

In [92]:
ucon2['values']['b2f3f52b-a703-49c0-9cdc-eaa399ccc81c']

"So you have all these methods of tracking intensity...how hard do you want to work? \nIt's a good idea to work at a variety of intensities: Low Intensity: This is about a Level 3-5 on the Perceived Exertion Scale. You might work at this intensity when you're warming up or if you're doing a longer workout, like a long bike ride, walk or run. This might also be an intensity you work at if you take walks throughout the day. Try this intensity about once a week.  Moderate Intensity: This is about a Level 5-6 on the Perceived Exertion Scale and where most of your workouts will fall. Think of getting on a cardio machine or going for a run and being at that place where you can talk, but only a few words. Try this level about 2-3 times a week.  High Intensity: This is about a Level 8-9 on the Perceived Exertion Scale and a level you can only work at for short periods of time. You might work at this intensity when doing high-intensity interval training. Shoot for one, maybe two times a week wi

In [93]:
index_p=[]
index_e=[]
index_l=[]
for i in range(0,len(ucon2)):
    if ucon2.type[i]=='PHYSICAL_FITNESS':
        index_p.append(ucon2.id[i])
    if ucon2.type[i]=='EMOTION':
        index_e.append(ucon2.id[i])
    if ucon2.type[i]=='LIFESTYLE':
        index_l.append(ucon2.id[i])
corr_p = pd.DataFrame(index = index_p)
corr_e = pd.DataFrame(index = index_e)
corr_l = pd.DataFrame(index = index_l)
for i in range(0,len(uques1)):
    if uques1.category[i]=='PHYSICAL_FITNESS':
        temp = uques1.details[i]
        corr_p[temp] = ''
    if uques1.category[i]=='EMOTION':
        temp = uques1.details[i]
        corr_e[temp] = ''
    if uques1.category[i]=='LIFESTYLE':
        temp = uques1.details[i]
        corr_l[temp] = ''

In [94]:
for i in corr_p.columns: #physical
    for j in corr_p.index:
        text1 = i
        text2 = ucon2['values'][j]

        vector1 = text_to_vector(text1)
        vector2 = text_to_vector(text2)

        cosine = get_cosine(vector1, vector2)
        corr_p[i][j]=cosine

In [95]:
for i in corr_e.columns: #emotional
    for j in corr_e.index:
        text1 = i
        text2 = ucon2['values'][j]

        vector1 = text_to_vector(text1)
        vector2 = text_to_vector(text2)

        cosine = get_cosine(vector1, vector2)
        corr_e[i][j]=cosine

In [96]:
for i in corr_l.columns: #lifestyle
    for j in corr_l.index:
        text1 = i
        text2 = ucon2['values'][j]

        vector1 = text_to_vector(text1)
        vector2 = text_to_vector(text2)

        cosine = get_cosine(vector1, vector2)
        corr_l[i][j]=cosine

In [97]:
corr_e

,"In the last month, how often have you found that you could not cope with all the things that you had to do ?","In the last month, how often have you felt difficulties were pilling up so high that you could not overcome them ?","In the last month, how often have you felt nervous and stressed ?","In the last month, how often have you felt confident about your ability to handle your personal problems ?","In the last month, how often have you been angered because of the things that were outside of your control ?","In the last month, how often have you felt that you were on top of things ?","In the last month, how often have you been able to control irritations in your life ?","In the last month, how often have you felt that you have not been able to control the important things of your life ?","In the last month, how often have you felt that things were going your way?",In the last one month how often have you been upset because of something that happened unexpectedly ?
1a5cb314-aa54-408f-a5e6-81db80e3e33c,0.232405,0.123724,0.304513,0.184277,0.322985,0.225325,0.21839,0.306068,0.144694,0.235855
253bf540-f194-421c-871d-d1d964ddaac2,0.208777,0.0946243,0.182466,0.123218,0.337445,0.210104,0.239057,0.273874,0.100432,0.220126
6c2d3398-0056-4c1b-9660-4f48752981f0,0.407994,0.25845,0.305763,0.307392,0.400213,0.366514,0.343674,0.460332,0.273482,0.295152
6dc81491-43c1-441f-9a0b-5da872c49aab,0.323237,0.171592,0.176645,0.287813,0.3015,0.283487,0.311234,0.348727,0.250615,0.189353
892d8c8c-e8a8-4be5-9cdc-de99f156cb57,0.429513,0.277028,0.26847,0.33952,0.368051,0.353412,0.322656,0.412361,0.338138,0.285402
c47b1867-91ea-4969-8947-5668f253d131,0.255,0.145309,0.246332,0.171727,0.357076,0.241355,0.291551,0.314274,0.168917,0.248353
c964ab5d-950f-4057-b2c7-f7956b27690d,0.357215,0.216518,0.256933,0.215605,0.358312,0.273886,0.271957,0.3581,0.217041,0.257841


In [98]:
corr_e['In the last one month how often have you been upset because of something that happened unexpectedly ?'].astype(str).astype(float).nlargest(10)

6c2d3398-0056-4c1b-9660-4f48752981f0    0.295152
892d8c8c-e8a8-4be5-9cdc-de99f156cb57    0.285402
c964ab5d-950f-4057-b2c7-f7956b27690d    0.257841
c47b1867-91ea-4969-8947-5668f253d131    0.248353
1a5cb314-aa54-408f-a5e6-81db80e3e33c    0.235855
253bf540-f194-421c-871d-d1d964ddaac2    0.220126
6dc81491-43c1-441f-9a0b-5da872c49aab    0.189353
Name: In the last one month how often have you been upset because of something that happened unexpectedly ?, dtype: float64

In [99]:
ucon2['values']['6c2d3398-0056-4c1b-9660-4f48752981f0']

"If your mood swings last for more than two weeks and don't seem to be getting any better, tell your practitioner and reach to a professional counsellor. Some of the best counsellors are listed in our Experts sections.  If you notice that you're frequently nervous or anxious, you may be suffering from some sort of anxiety disorder. Finally, if your mood swings become more frequent and intense, you may have a condition called bipolar disorder, in which you may swing from periods of depression to mania.Accurate information about depression during pregnancy or following the birth of your baby that lets you know that you are not alone and that help is available. The life of every mother is different. No two mothers would have the same changes, the same family structure, the same family support and so on. So the challenges of each mother is different and hence is it important to have a solution that works for you. Professional support is important as it ensures that you are understood well 

In [100]:
ucon2['values']['892d8c8c-e8a8-4be5-9cdc-de99f156cb57']

'Clueing your partner in about how you\'re feeling and reassuring him that you still love him will help him avoid taking your outbursts personally. Make sure you\'re spending plenty of time together and nurturing your relationship. Go on a vacation if you can. Strengthen your connection now so you can really be there for one another after the baby comes. Air your worries about the future with understanding friends. Just putting your concerns into words often helps dissipate them or gives you insight into solutions. Keep the lines of communication between you and your partner free and clear, too. Make it a two-way street. In addition to pouring out your feelings, let him express his own.  Rather than let frustration build up in your life, find ways to decompress. Identify sources of stress in your life and change what you can, such as trimming your "to-do" list. If you still find anxiety creeping in, try taking a pregnancy yoga class, practicing meditation or other relaxation techniques

In [101]:
corr_l

,"Does your usual diet consist of Rawas (a type of fish), Shrimp or Prawns?",How much sleep do you get in a night?,"Do you frequently eat Flaxseeds, Chia seeds, Walnuts or Edamame ?",Does your usual diet consist of Eggs ?,What is your general diet ?,Does your usual diet consist of Paneer ?,Does your usual diet consist of dal/pulses ?,Do you consume milk/dairy products daily ?,Do you have a fruit every day ?,How much water do you drink in a day?,How often do you smoke / consume alcohol ?,"Does your usual diet consist of Meat (eg. Chicken, Mutton, Pork, etc) ?","Does your usual diet consist of nuts (eg. Almonds, Cashews, Peanuts, etc.) ?",Does your daily diet include green leafy vegetables ?,Do you often include dessert/ sweet dishes in your diet ?
00310b32-dda8-44ac-b952-81fdeac3862e,0.351986,0.20584,0.0468664,0.224064,0.0883722,0.224064,0.209593,0.018672,0.158712,0.197606,0.00933601,0.171132,0.171132,0.0873304,0.164032
015d6cbb-b214-47af-9d5b-88fc1c5a3708,0.275345,0.0734254,0,0.197734,0.0861967,0.197734,0.184964,0.0416283,0.0208141,0.0734254,0,0.151022,0.151022,0,0.0522431
038d3ef4-40e3-4f3a-b801-9eeb6f5357f4,0.194372,0.158226,0.0465846,0.117545,0.120781,0.117545,0.109953,0.0340262,0.0866122,0.158226,0.0494927,0.0897766,0.0897766,0.0347221,0.126814
064aa5bc-3d41-41d9-ae38-eea0aeb7a90e,0.319778,0.213185,0.137527,0.290075,0.274577,0.290075,0.27134,0.145038,0.241729,0.238767,0.145038,0.221549,0.221549,0.162804,0.291233
06c7e731-ffc7-4635-8d85-c163c2b1ba69,0.213797,0.213797,0.0193167,0.121211,0.0341474,0.121211,0.113383,0,0.0923514,0.117079,0.0173159,0.0925766,0.0925766,0.0107984,0.0627792
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7b864ed9-c589-4164-84f2-9399f5d48bce,0.210736,0.0561964,0.0228482,0.163853,0.0969368,0.163853,0.178816,0.0273089,0.0182059,0.104365,0,0.125145,0.125145,0.0766353,0.0837769
7bf74c49-2b36-4c42-92c5-553db02d9815,0.274204,0.223933,0.0563621,0.181369,0.141022,0.181369,0.169655,0.0362738,0.150277,0.242214,0.0362738,0.138523,0.142481,0.0727094,0.199435
81ad847b-a227-4a02-80ca-d3216dbe8a1c,0.206703,0.225494,0.0396152,0.113638,0.145663,0.113638,0.106299,0.0378794,0.142048,0.208791,0.0378794,0.0867926,0.101258,0.0265747,0.134692
86895125-806b-4909-b198-bfaf25943dd7,0.272661,0.185465,0.0245101,0.173678,0.115129,0.173678,0.162461,0.0303414,0.109857,0.182697,0.0177863,0.134247,0.134247,0.0352325,0.138307


In [102]:
corr_l['How much water do you drink in a day?'].astype(str).astype(float).nlargest(10)

6fd4e033-3aae-4a27-9a1c-be3345be834b    0.379942
09e718a8-ad3e-4b3f-9868-c2eacc315632    0.285311
581a5991-121a-444e-81d4-6bdd9c3cee7a    0.257423
5bc1b7c9-2021-4f90-b9a7-b2cf88faae66    0.247246
6d62c06e-ec7b-40d4-8cc9-d2ac20411586    0.242635
33181948-61ca-4db1-b61f-de63c995b274    0.242281
7bf74c49-2b36-4c42-92c5-553db02d9815    0.242214
064aa5bc-3d41-41d9-ae38-eea0aeb7a90e    0.238767
186aff91-a875-49f8-8451-17be361b8178    0.237641
44e09a2a-128f-4c30-86b2-7c350886fd1d    0.218497
Name: How much water do you drink in a day?, dtype: float64

In [103]:
ucon2['values']['6fd4e033-3aae-4a27-9a1c-be3345be834b']

'It’s common to get a little constipated if you don’t drink enough water. Inactivity, diet changes, illness, and even stress can add to the problem. The constipation usually passes on its own, but you can help move things along with exercise, over-the-counter meds, and of course, water. See a doctor if your constipation lasts for more than a week or two or you have dizziness, bad pain in your belly, or blood in your stool. These could be signs of something more serious. Water makes up a large part of your joint cartilage that helps absorb shock and make bone-against-bone movements smoother. Water also can help keep gout (a painful joint condition) at bay.  It helps flush toxins from your body that could inflame your joints, too. And that’s a good thing! As sweat evaporates from your skin, it cools down your body. Have a couple of extra cups of water in the couple of hours before you head out for exercise. Try to take about 10 big gulps every 15 minutes or so during your workout, too. R

In [104]:
corr_p

,How much time do you spend standing or walking during the day ?,How much time do you spend sitting on the computer/ watching TV/ talking on the phone ?,"Do you feel overly short of breath, dizzy, nauseous or feel overexerted after exercise ?",Do you participate in Spinning/ Cardio/ Zumba sessions every week ?,How much time do you spend sitting at a place during the day ?,Do you exercise at least every alternate day ?,"Do you participate in some outdoor sports (Badminton, Tennis, Swimming, Running, etc) every week ?",Do you participate in Pilates/ Yoga sessions every week ?,"Do you do the chores(cleaning, cooking or laundry) at home and if so, for how long ?"
19488ef8-8135-4770-8379-549a166d8114,0.265684,0.222948,0.210915,0.0970143,0.297805,0.0813489,0.081992,0.127827,0.186016
28f88e8d-a4f7-423a-9716-8e2cc163cf8d,0.299727,0.273138,0.268733,0.15451,0.277805,0.172748,0.143644,0.162868,0.337737
39aac870-13d0-4d93-8285-c46ee2649520,0.298108,0.162998,0.227796,0.115257,0.219022,0.150338,0.113645,0.121491,0.265194
9ae77239-924a-49c6-8dd8-1cc7128e8210,0.258517,0.327676,0.188992,0.136079,0.267729,0.12747,0.118116,0.15507,0.313104
9bcfb27f-5b46-40e9-8cc6-e0a675356bc4,0.171241,0.18753,0.0920478,0.0970269,0.249622,0.0506237,0.0820027,0.102275,0.238132
a2cbc9ed-80fd-4eaa-83db-5571e7432386,0.126745,0.147264,0.0548821,0.0347105,0.152216,0.0776151,0.0293357,0.0365881,0.133109
a386f691-f464-450f-a532-bd7f27498c57,0.149441,0.173634,0.165369,0.0909467,0.247273,0.11185,0.076864,0.0958662,0.181358
b2f3f52b-a703-49c0-9cdc-eaa399ccc81c,0.193113,0.216067,0.167241,0.14103,0.371075,0.197096,0.119192,0.148659,0.28844
b3b9959e-0729-43bb-962b-49140c9d84c5,0.211578,0.245831,0.187396,0.068478,0.226377,0.106007,0.0667782,0.0721821,0.266641
e746ecb0-2b6b-4cb6-9148-c59ee07630a3,0.225317,0.23023,0.232495,0.162798,0.25793,0.182013,0.150904,0.171604,0.302081


In [105]:
corr_p['How much time do you spend standing or walking during the day ?'].astype(str).astype(float).nlargest(10)

28f88e8d-a4f7-423a-9716-8e2cc163cf8d    0.299727
39aac870-13d0-4d93-8285-c46ee2649520    0.298108
19488ef8-8135-4770-8379-549a166d8114    0.265684
9ae77239-924a-49c6-8dd8-1cc7128e8210    0.258517
e746ecb0-2b6b-4cb6-9148-c59ee07630a3    0.225317
e9f5480c-145c-411c-8e99-7fce98070a11    0.215018
b3b9959e-0729-43bb-962b-49140c9d84c5    0.211578
b2f3f52b-a703-49c0-9cdc-eaa399ccc81c    0.193113
9bcfb27f-5b46-40e9-8cc6-e0a675356bc4    0.171241
a386f691-f464-450f-a532-bd7f27498c57    0.149441
Name: How much time do you spend standing or walking during the day ?, dtype: float64

In [106]:
ucon2['values']['39aac870-13d0-4d93-8285-c46ee2649520']

"Exercising during pregnancy does the average pregnant mom and her baby good. However certain conditions can make exercise during pregnancy risky.  Anemia,   chronic bronchitis,   poorly controlled type 1 diabetes,   being obese or extremely underweight (BMI under 12),   a history of an extremely sedentary lifestyle,   poorly controlled hypertension,   incompetent cervix,   preterm rupture of placenta membranes,  persistent vaginal bleeding during the second or third trimester,   seizure disorders,   poorly controlled hyperthyroidism, and   intrauterine growth restriction may warrant skipping some or all workouts during pregnancy.  That's why it's essential to always get the green light from your doctor for any exercise program. If your practitioner has restricted exercise for you during part or all of your pregnancy, ask if there are any workouts you can work in (say, arm-only workouts or stretching) to help you stay in shape, even if you're on modified bed rest. "

In [107]:
ucon2['values']['39aac870-13d0-4d93-8285-c46ee2649520']

"Exercising during pregnancy does the average pregnant mom and her baby good. However certain conditions can make exercise during pregnancy risky.  Anemia,   chronic bronchitis,   poorly controlled type 1 diabetes,   being obese or extremely underweight (BMI under 12),   a history of an extremely sedentary lifestyle,   poorly controlled hypertension,   incompetent cervix,   preterm rupture of placenta membranes,  persistent vaginal bleeding during the second or third trimester,   seizure disorders,   poorly controlled hyperthyroidism, and   intrauterine growth restriction may warrant skipping some or all workouts during pregnancy.  That's why it's essential to always get the green light from your doctor for any exercise program. If your practitioner has restricted exercise for you during part or all of your pregnancy, ask if there are any workouts you can work in (say, arm-only workouts or stretching) to help you stay in shape, even if you're on modified bed rest. "

In [108]:
corr_p['Do you feel overly short of breath, dizzy, nauseous or feel overexerted after exercise ?'].astype(str).astype(float).nlargest(10)

28f88e8d-a4f7-423a-9716-8e2cc163cf8d    0.268733
e746ecb0-2b6b-4cb6-9148-c59ee07630a3    0.232495
39aac870-13d0-4d93-8285-c46ee2649520    0.227796
19488ef8-8135-4770-8379-549a166d8114    0.210915
9ae77239-924a-49c6-8dd8-1cc7128e8210    0.188992
b3b9959e-0729-43bb-962b-49140c9d84c5    0.187396
e9f5480c-145c-411c-8e99-7fce98070a11    0.186211
b2f3f52b-a703-49c0-9cdc-eaa399ccc81c    0.167241
a386f691-f464-450f-a532-bd7f27498c57    0.165369
9bcfb27f-5b46-40e9-8cc6-e0a675356bc4    0.092048
Name: Do you feel overly short of breath, dizzy, nauseous or feel overexerted after exercise ?, dtype: float64

In [109]:
ucon2['values']['28f88e8d-a4f7-423a-9716-8e2cc163cf8d']

'Pregnant women who exercise should monitor signs of potential problems, keeping in mind that if there\'s something wrong with the pregnancy, it\'s not caused by exercise. Rather, exercise might provide that extra bit of stress that brings the problem to the forefront. Stop exercising and consult your doctor if you have: Vaginal bleeding  Abdominal cramping  Light-headedness  Excessive nausea  Extreme headaches Also watch out for dehydration, which can cause preterm uterine contractions, and in the worst-case scenario, lead to preterm labor. Eight or nine cups of water a day is recommended during pregnancy, on average, but fluid needs should be adjusted for intensity and duration of the workout. If you experience any of these symptoms, don’t try to push through it, since doing so can leave you more susceptible to injuries: Excessive fatigue. While a good workout might leave you feeling a bit tired but accomplished, it shouldn’t exhaust you so much that walking to your car in the gym pa

In [110]:
act

,modifiedAt,userId,status,createdAt,details,id,createdBy,modifiedBy,type
0,1620825813342,efafd1a8-fb7e-453e-8b14-26741ceb33d4,ACTIVE,1620825813342,"{'daysFromLMP': Decimal('200'), 'tasks': [{'ti...",13116045-40f0-42af-9803-d1a518eebad5,SYSTEM,SYSTEM,DOCTOR_TODO
1,1620825813342,efafd1a8-fb7e-453e-8b14-26741ceb33d4,ACTIVE,1620825813342,"{'daysFromLMP': Decimal('248'), 'tasks': [{'ti...",1e599aa8-da23-40bf-8cbe-bf4cb686ccd1,SYSTEM,SYSTEM,DOCTOR_TODO
2,1620825813342,efafd1a8-fb7e-453e-8b14-26741ceb33d4,ACTIVE,1620825813342,"{'daysFromLMP': Decimal('87'), 'tasks': [{'des...",4ec3850a-1fc9-4c7b-898e-027023a4624e,SYSTEM,SYSTEM,DOCTOR_TODO
3,1620825813342,efafd1a8-fb7e-453e-8b14-26741ceb33d4,ACTIVE,1620825813342,"{'daysFromLMP': Decimal('115'), 'tasks': [{'de...",5f26516d-90ce-467c-b2d0-645dc1b00382,SYSTEM,SYSTEM,DOCTOR_TODO
4,1620825813342,efafd1a8-fb7e-453e-8b14-26741ceb33d4,ACTIVE,1620825813342,"{'daysFromLMP': Decimal('185'), 'tasks': [{'ti...",638d45df-e13a-4bce-84a3-1154f684ff20,SYSTEM,SYSTEM,DOCTOR_TODO
...,...,...,...,...,...,...,...,...,...
1960,1622219354557,e502481d-93a0-4238-a320-0381c3904f8d,ACTIVE,1622219354557,"{'daysFromLMP': Decimal('241'), 'tasks': [{'de...",38f57469-730c-46e4-a13f-8fc5a8bb5f00,SYSTEM,SYSTEM,DOCTOR_TODO
1961,1622219354557,e502481d-93a0-4238-a320-0381c3904f8d,ACTIVE,1622219354557,"{'daysFromLMP': Decimal('136'), 'tasks': [{'de...",7e147710-09da-4e60-a87f-b7d0622a3852,SYSTEM,SYSTEM,DOCTOR_TODO
1962,1622219354557,e502481d-93a0-4238-a320-0381c3904f8d,ACTIVE,1622219354557,"{'daysFromLMP': Decimal('185'), 'tasks': [{'ti...",82e986ba-bcc1-4b9e-9350-9f108346f508,SYSTEM,SYSTEM,DOCTOR_TODO
1963,1622219354557,e502481d-93a0-4238-a320-0381c3904f8d,ACTIVE,1622219354557,"{'daysFromLMP': Decimal('248'), 'tasks': [{'ti...",8f5938ad-80bc-4e9b-99d4-c3baadc5bc16,SYSTEM,SYSTEM,DOCTOR_TODO


In [111]:
uuser = act.drop(columns = ['modifiedAt','createdAt','createdBy','modifiedBy'])
uuser = uuser[uuser.type == 'DOCTOR_TODO']

In [112]:
useruid= uuser.userId.unique()

In [113]:
uusers_c = uusers.copy()
for i in uusers_c.index: #dropping NaN values
    if uusers_c['lastMenstrualDate'][i] == 'NaN-NaN-NaN':
        uusers_c = uusers_c.drop(i)
uusers_c = uusers_c.dropna()

In [114]:
from datetime import date
import datetime
def trimester(days):
    if days in range(0,85):
        return 1
    if days in range(85,190):
        return 2
    if days in range(190,350):
        return 3
def week_c(days):
    day = math.ceil(days/7)
    return day
def day_calc(dat):
    date_format = "%Y-%m-%d"
    today = date.today()
    dat = datetime.datetime.strptime(dat, "%Y-%m-%d").date()
    number = today - dat
    return number.days

In [115]:
uusers_c['week'] = ''
uusers_c['trimester'] = ''
uusers_c['day'] = ''
for i in uusers_c.index:
    temp = uusers_c['lastMenstrualDate'][i]
    uusers_c['day'][i] = day_calc(temp)
    uusers_c['week'][i] = week_c(uusers_c['day'][i])
    uusers_c['trimester'][i] = trimester(uusers_c['day'][i])

In [116]:
uusers_c

,lastName,firstName,userId,leapFavourites,weight,dateOfBirth,height,lastMenstrualDate,week,trimester,day
18,Jain,Neha,a01814b7-0f02-4159-b766-2b64a05195a8,[],"{'value': '75', 'units': 'kgs'}",1991-05-29,"{'value': Decimal('64'), 'units': 'inches'}",2020-11-01,34,3,236
23,Sheikh,Saif,95ff66c9-a491-4921-9e59-56d108dc6dc7,[],"{'value': '65.65', 'units': 'kgs'}",2000-05-31,"{'value': Decimal('71'), 'units': 'inches'}",2021-03-28,13,2,89
28,Shenoy,Vinayak,a973c169-82d7-4b05-a1c0-a5c3dbccac51,[],"{'value': '80', 'units': 'kgs'}",1982-01-28,"{'value': Decimal('67'), 'units': 'inches'}",2020-09-10,42,3,288
42,VN,Manasi,3410d897-7813-4b3e-a6a7-685d1fb231e4,[],"{'value': '72.3', 'units': 'kgs'}",2003-12-18,"{'value': Decimal('65'), 'units': 'inches'}",2021-03-17,15,2,100
50,Sheikh,Saif,263b167d-f79d-4539-bcf8-bb41dd477d90,[],"{'value': '69.69', 'units': 'kgs'}",2000-04-29,"{'value': Decimal('71'), 'units': 'inches'}",2021-03-02,17,2,115
...,...,...,...,...,...,...,...,...,...,...,...
1989,punjabi,Sunil,a9cfc823-61bd-4c32-9b71-4433f393badc,"[{'type': 'LIFESTYLE', 'id': '620823ad-df2e-41...","{'value': '60', 'units': 'kgs'}",2000-07-10,"{'value': Decimal('70'), 'units': 'inches'}",2021-04-08,12,1,78
1994,Test,Firoz-test,7776f9a6-75c4-4847-89d3-a94b74167bd2,[],"{'value': '55.5', 'units': 'kgs'}",2004-06-01,"{'value': Decimal('65'), 'units': 'inches'}",2021-05-22,5,1,34
1998,Jain,Vishal,b3353cc5-3a57-46e8-9733-b38d6f94fb42,[],"{'value': '70.00', 'units': 'kgs'}",2001-08-01,"{'value': Decimal('68'), 'units': 'inches'}",2021-05-26,5,1,30
2007,Sheikh,Saif,809af84a-c49b-44c3-9426-ecdcc40210b6,[],"{'value': '100.0', 'units': 'kgs'}",1995-06-08,"{'value': Decimal('72'), 'units': 'inches'}",2021-05-24,5,1,32


In [117]:
uact1.questions[17]

'How much sleep do you get in a night? Does your usual diet consist of nuts (eg. Almonds, Cashews, Peanuts, etc.) ? Does your usual diet consist of Rawas (a type of fish), Shrimp or Prawns? Do you participate in Pilates/ Yoga sessions every week ? Does your usual diet consist of Eggs ? Does your daily diet include green leafy vegetables ? Do you often include dessert/ sweet dishes in your diet ? Do you participate in some outdoor sports (Badminton, Tennis, Swimming, Running, etc) every week ? Do you frequently eat Flaxseeds, Chia seeds, Walnuts or Edamame ? Do you feel overly short of breath, dizzy, nauseous or feel overexerted after exercise ? Do you participate in Spinning/ Cardio/ Zumba sessions every week ? Do you exercise at least every alternate day ? Does your usual diet consist of Meat (eg. Chicken, Mutton, Pork, etc) ? Does your usual diet consist of dal/pulses ? Do you have a fruit every day ? In the last month, how often have you felt that you have not been able to control t

In [118]:
import datetime
datetime.datetime.strptime('2020-11-01', "%Y-%m-%d").date()

datetime.date(2020, 11, 1)

In [119]:
uusers_c

,lastName,firstName,userId,leapFavourites,weight,dateOfBirth,height,lastMenstrualDate,week,trimester,day
18,Jain,Neha,a01814b7-0f02-4159-b766-2b64a05195a8,[],"{'value': '75', 'units': 'kgs'}",1991-05-29,"{'value': Decimal('64'), 'units': 'inches'}",2020-11-01,34,3,236
23,Sheikh,Saif,95ff66c9-a491-4921-9e59-56d108dc6dc7,[],"{'value': '65.65', 'units': 'kgs'}",2000-05-31,"{'value': Decimal('71'), 'units': 'inches'}",2021-03-28,13,2,89
28,Shenoy,Vinayak,a973c169-82d7-4b05-a1c0-a5c3dbccac51,[],"{'value': '80', 'units': 'kgs'}",1982-01-28,"{'value': Decimal('67'), 'units': 'inches'}",2020-09-10,42,3,288
42,VN,Manasi,3410d897-7813-4b3e-a6a7-685d1fb231e4,[],"{'value': '72.3', 'units': 'kgs'}",2003-12-18,"{'value': Decimal('65'), 'units': 'inches'}",2021-03-17,15,2,100
50,Sheikh,Saif,263b167d-f79d-4539-bcf8-bb41dd477d90,[],"{'value': '69.69', 'units': 'kgs'}",2000-04-29,"{'value': Decimal('71'), 'units': 'inches'}",2021-03-02,17,2,115
...,...,...,...,...,...,...,...,...,...,...,...
1989,punjabi,Sunil,a9cfc823-61bd-4c32-9b71-4433f393badc,"[{'type': 'LIFESTYLE', 'id': '620823ad-df2e-41...","{'value': '60', 'units': 'kgs'}",2000-07-10,"{'value': Decimal('70'), 'units': 'inches'}",2021-04-08,12,1,78
1994,Test,Firoz-test,7776f9a6-75c4-4847-89d3-a94b74167bd2,[],"{'value': '55.5', 'units': 'kgs'}",2004-06-01,"{'value': Decimal('65'), 'units': 'inches'}",2021-05-22,5,1,34
1998,Jain,Vishal,b3353cc5-3a57-46e8-9733-b38d6f94fb42,[],"{'value': '70.00', 'units': 'kgs'}",2001-08-01,"{'value': Decimal('68'), 'units': 'inches'}",2021-05-26,5,1,30
2007,Sheikh,Saif,809af84a-c49b-44c3-9426-ecdcc40210b6,[],"{'value': '100.0', 'units': 'kgs'}",1995-06-08,"{'value': Decimal('72'), 'units': 'inches'}",2021-05-24,5,1,32


In [120]:
for i in ucon2.index: #remove all the .decimal object
    ucon2['weeks'][i] = int(eval(ucon2['weeks'][i])[0])
    ucon2['trimesters'][i] = int(eval(ucon2['trimesters'][i])[0])

In [121]:
ucon2

,status,subtype,weeks,trimesters,data,category,id,tags,type,values,clean,Key_words
id,,,,,,,,,,,,
19488ef8-8135-4770-8379-549a166d8114,ACTIVE,PHYSICAL_FITNESS,4,1,"{'title': 'TRIMESTER TIPS ', 'featureMedia': {...",LEAP,19488ef8-8135-4770-8379-549a166d8114,"['Pregnancy', 'Yoga', 'Exercise', 'Pilates ', ...",PHYSICAL_FITNESS,You can pretty much continue doing exactly wha...,pretty much continue exactly pregnancy include...,exactly feel sluggish drop carry throughout 30...
28f88e8d-a4f7-423a-9716-8e2cc163cf8d,ACTIVE,PHYSICAL_FITNESS,6,1,{'title': 'Warning Signs: Working Out While Pr...,LEAP,28f88e8d-a4f7-423a-9716-8e2cc163cf8d,"['Child Development', 'Pregnancy', 'Exercise',...",PHYSICAL_FITNESS,Pregnant women who exercise should monitor sig...,pregnant women exercise monitor sign potential...,encourage expectant moms gynecologists wait of...
39aac870-13d0-4d93-8285-c46ee2649520,ACTIVE,PHYSICAL_FITNESS,4,1,{'title': 'WHEN YOU SHOULDN’T EXERCISE AT ALL ...,LEAP,39aac870-13d0-4d93-8285-c46ee2649520,"['Exercise', 'Pregnancy', 'Anemia', 'Hypertens...",PHYSICAL_FITNESS,Exercising during pregnancy does the average p...,exercise pregnancy average pregnant mom baby g...,poorly controlled type 1 diabetes ask intraute...
9ae77239-924a-49c6-8dd8-1cc7128e8210,ACTIVE,PHYSICAL_FITNESS,5,1,"{'title': 'BEST EXERCISES DURING PREGNANCY. ',...",LEAP,9ae77239-924a-49c6-8dd8-1cc7128e8210,"['Pregnancy', 'Child Development', 'Pilates', ...",PHYSICAL_FITNESS,If you have access to prenatal exercise classe...,access prenatal exercise class sign workouts m...,emphasis good time pregnant ph motion may feel...
9bcfb27f-5b46-40e9-8cc6-e0a675356bc4,ACTIVE,PHYSICAL_FITNESS,7,1,"{'title': 'PRENATAL YOGA', 'featureMedia': {'i...",LEAP,9bcfb27f-5b46-40e9-8cc6-e0a675356bc4,"['Pregnancy', 'Child Development', 'Yoga', 'Ex...",PHYSICAL_FITNESS,Prenatal yoga sounds like a healthy practice t...,prenatal yoga sound like healthy practice adop...,relax fear deep prenatal yoga healthy baby bre...
...,...,...,...,...,...,...,...,...,...,...,...,...
7b864ed9-c589-4164-84f2-9399f5d48bce,ACTIVE,LIFESTYLE,14,2,"{'title': 'FIBRE', 'featureMedia': {'image': '...",LEAP,7b864ed9-c589-4164-84f2-9399f5d48bce,"['Child Development', 'Pregnancy', 'Nutrition'...",LIFESTYLE,This is plant material like carbohydrates and ...,plant material like carbohydrates substances r...,mostly consisting disorders 1g per 100g whole ...
7bf74c49-2b36-4c42-92c5-553db02d9815,ACTIVE,LIFESTYLE,7,1,"{'title': 'Bananas', 'featureMedia': {'image':...",LEAP,7bf74c49-2b36-4c42-92c5-553db02d9815,"['Pregnancy', 'Child Developments', 'Bananas',...",LIFESTYLE,Bananas are rich in potassium and offer quick ...,bananas rich potassium offer quick energy figh...,easily available fruit helps gas yoghurt bones...
81ad847b-a227-4a02-80ca-d3216dbe8a1c,ACTIVE,LIFESTYLE,9,1,"{'title': 'Basil', 'featureMedia': {'image': '...",LEAP,81ad847b-a227-4a02-80ca-d3216dbe8a1c,"['Pregnancy', 'Child Development', 'Basil', 'I...",LIFESTYLE,"Basil is also packed with iron, vital for keep...",basil also pack iron vital keep energy level c...,nutrients phosphorus oxidizing cholesterol fre...


In [122]:
uact1.details[17]

{'answers': {'How much sleep do you get in a night?': '1',
  'Does your usual diet consist of nuts (eg. Almonds, Cashews, Peanuts, etc.) ?': '1',
  'Does your usual diet consist of Rawas (a type of fish), Shrimp or Prawns?': '1',
  'Do you participate in Pilates/ Yoga sessions every week ?': '1',
  'Does your usual diet consist of Eggs ?': '1',
  'Does your daily diet include green leafy vegetables ?': '1',
  'Do you often include dessert/ sweet dishes in your diet ?': '1',
  'Do you participate in some outdoor sports (Badminton, Tennis, Swimming, Running, etc) every week ?': '1',
  'Do you frequently eat Flaxseeds, Chia seeds, Walnuts or Edamame ?': '1',
  'Do you feel overly short of breath, dizzy, nauseous or feel overexerted after exercise ?': '1',
  'Do you participate in Spinning/ Cardio/ Zumba sessions every week ?': '2',
  'Do you exercise at least every alternate day ?': '2',
  'Does your usual diet consist of Meat (eg. Chicken, Mutton, Pork, etc) ?': '2',
  'Does your usual d

In [123]:
df_recc = pd.DataFrame()

In [124]:
corr_p

,How much time do you spend standing or walking during the day ?,How much time do you spend sitting on the computer/ watching TV/ talking on the phone ?,"Do you feel overly short of breath, dizzy, nauseous or feel overexerted after exercise ?",Do you participate in Spinning/ Cardio/ Zumba sessions every week ?,How much time do you spend sitting at a place during the day ?,Do you exercise at least every alternate day ?,"Do you participate in some outdoor sports (Badminton, Tennis, Swimming, Running, etc) every week ?",Do you participate in Pilates/ Yoga sessions every week ?,"Do you do the chores(cleaning, cooking or laundry) at home and if so, for how long ?"
19488ef8-8135-4770-8379-549a166d8114,0.265684,0.222948,0.210915,0.0970143,0.297805,0.0813489,0.081992,0.127827,0.186016
28f88e8d-a4f7-423a-9716-8e2cc163cf8d,0.299727,0.273138,0.268733,0.15451,0.277805,0.172748,0.143644,0.162868,0.337737
39aac870-13d0-4d93-8285-c46ee2649520,0.298108,0.162998,0.227796,0.115257,0.219022,0.150338,0.113645,0.121491,0.265194
9ae77239-924a-49c6-8dd8-1cc7128e8210,0.258517,0.327676,0.188992,0.136079,0.267729,0.12747,0.118116,0.15507,0.313104
9bcfb27f-5b46-40e9-8cc6-e0a675356bc4,0.171241,0.18753,0.0920478,0.0970269,0.249622,0.0506237,0.0820027,0.102275,0.238132
a2cbc9ed-80fd-4eaa-83db-5571e7432386,0.126745,0.147264,0.0548821,0.0347105,0.152216,0.0776151,0.0293357,0.0365881,0.133109
a386f691-f464-450f-a532-bd7f27498c57,0.149441,0.173634,0.165369,0.0909467,0.247273,0.11185,0.076864,0.0958662,0.181358
b2f3f52b-a703-49c0-9cdc-eaa399ccc81c,0.193113,0.216067,0.167241,0.14103,0.371075,0.197096,0.119192,0.148659,0.28844
b3b9959e-0729-43bb-962b-49140c9d84c5,0.211578,0.245831,0.187396,0.068478,0.226377,0.106007,0.0667782,0.0721821,0.266641
e746ecb0-2b6b-4cb6-9148-c59ee07630a3,0.225317,0.23023,0.232495,0.162798,0.25793,0.182013,0.150904,0.171604,0.302081


In [125]:
uact1['details'][17]['answers'].keys()

dict_keys(['How much sleep do you get in a night?', 'Does your usual diet consist of nuts (eg. Almonds, Cashews, Peanuts, etc.) ?', 'Does your usual diet consist of Rawas (a type of fish), Shrimp or Prawns?', 'Do you participate in Pilates/ Yoga sessions every week ?', 'Does your usual diet consist of Eggs ?', 'Does your daily diet include green leafy vegetables ?', 'Do you often include dessert/ sweet dishes in your diet ?', 'Do you participate in some outdoor sports (Badminton, Tennis, Swimming, Running, etc) every week ?', 'Do you frequently eat Flaxseeds, Chia seeds, Walnuts or Edamame ?', 'Do you feel overly short of breath, dizzy, nauseous or feel overexerted after exercise ?', 'Do you participate in Spinning/ Cardio/ Zumba sessions every week ?', 'Do you exercise at least every alternate day ?', 'Does your usual diet consist of Meat (eg. Chicken, Mutton, Pork, etc) ?', 'Does your usual diet consist of dal/pulses ?', 'Do you have a fruit every day ?', 'In the last month, how ofte

In [182]:
uact1

,userId,status,details,id,type,questions,Key_words,recommend
17,95ff66c9-a491-4921-9e59-56d108dc6dc7,ACTIVE,{'answers': {'How much sleep do you get in a n...,2616ef0d-681c-4efd-8036-de872a5bab81,LEAP_SCORE,How much sleep do you get in a night? Does you...,"fish ), shrimp felt nervous often include dess...","[09e718a8-ad3e-4b3f-9868-c2eacc315632, 6fd4e03..."
48,263b167d-f79d-4539-bcf8-bb41dd477d90,ACTIVE,{'answers': {'Do you exercise at least every a...,09459b12-7a3b-4285-8421-6e7c09d806d8,LEAP_SCORE,Do you exercise at least every alternate day ?...,"fish ), shrimp felt nervous often include dess...","[b2f3f52b-a703-49c0-9cdc-eaa399ccc81c, e746ecb..."
76,210b6961-7162-48d8-b994-b9425d3d2f55,ACTIVE,"{'answers': {'In the last month, how often hav...",3022a4d6-8f12-4270-8a3a-19ca7b2d809e,LEAP_SCORE,"In the last month, how often have you been ang...","fish ), shrimp felt nervous often include dess...","[6c2d3398-0056-4c1b-9660-4f48752981f0, 892d8c8..."
94,44de4bd9-bd05-4b7b-bc82-c7ff66d779a3,ACTIVE,{'answers': {'How much water do you drink in a...,9a21a368-a0ae-42fd-9359-3981cad3f3c6,LEAP_SCORE,How much water do you drink in a day?,day much water drink,[]
103,e5b74707-e438-4eb7-a75e-aa84b5647f75,ACTIVE,{'answers': {'What is your general diet ?': '1...,5abcf36b-03ab-4345-93c4-cb33bc8b4ae7,LEAP_SCORE,What is your general diet ? Does your usual di...,"fish ), shrimp meat edamame cashews felt nervo...","[064aa5bc-3d41-41d9-ae38-eea0aeb7a90e, 78d31c7..."
...,...,...,...,...,...,...,...,...
1825,b59bed52-deff-42a9-9431-ce1207682897,ACTIVE,{'answers': {'How much sleep do you get in a n...,ad647704-86ff-4ebf-9ec6-af6af2ae0256,LEAP_SCORE,How much sleep do you get in a night? Does you...,"fish ), shrimp rawas prawns type get night gen...","[09e718a8-ad3e-4b3f-9868-c2eacc315632, 6fd4e03..."
1865,d2b3a8a0-b1e3-4bd1-b0b6-a2067445739e,ACTIVE,{'answers': {'Does your usual diet consist of ...,2f233950-cb0f-4111-b121-10b768dcbc99,LEAP_SCORE,Does your usual diet consist of Rawas (a type ...,"fish ), shrimp felt nervous often include dess...","[78d31c7e-b2c2-4d55-8172-761233226bdd, 186aff9..."
1905,9af175e6-735f-42ba-b339-917a7a7ba0a7,ACTIVE,{'answers': {'How much sleep do you get in a n...,e01e1b99-5346-4714-acb4-47a9ab79cc3d,LEAP_SCORE,How much sleep do you get in a night? How much...,get drink night day much water sleep,"[09e718a8-ad3e-4b3f-9868-c2eacc315632, 6fd4e03..."
1938,ffde9a43-1abf-46d0-bb3a-3bb349e49ea8,ACTIVE,{'answers': {'How often do you smoke / consume...,bf18d364-0c3d-41db-9dd9-b1765e4f91dd,LEAP_SCORE,How often do you smoke / consume alcohol ? In ...,stressed felt nervous something personal probl...,"[09e718a8-ad3e-4b3f-9868-c2eacc315632, 6fd4e03..."


In [164]:
uact1['recommend'] =''
for i in uact1.index:
    lst = []
    que = list(uact1['details'][i]['answers'].keys())[0:int((0.5*len(uact1['details'][i]['answers'])))]
    for j in range(0,int((0.5*len(uact1['details'][i]['answers'])))):
        if que[j] in corr_p.columns:
            lst.append(corr_p[que[j]].astype(str).astype(float).nlargest(3))
        if que[j] in corr_l.columns:
            lst.append(corr_l[que[j]].astype(str).astype(float).nlargest(3))
        if que[j] in corr_e.columns:
            lst.append(corr_e[que[j]].astype(str).astype(float).nlargest(3))
        
        uact1['recommend'][i] = lst

In [146]:
corr_l[que[0]].astype(str).astype(float).nlargest(3).keys()

Index(['09e718a8-ad3e-4b3f-9868-c2eacc315632',
       '6fd4e033-3aae-4a27-9a1c-be3345be834b',
       '6d62c06e-ec7b-40d4-8cc9-d2ac20411586'],
      dtype='object')

In [147]:
corr_p.columns

Index(['How much time do you spend standing or walking during the day ?',
       'How much time do you spend sitting on the computer/ watching TV/ talking on the phone ?',
       'Do you feel overly short of breath, dizzy, nauseous or feel overexerted after exercise ?',
       'Do you participate in Spinning/ Cardio/ Zumba sessions every week ?',
       'How much time do you spend sitting at a place during the day ?',
       'Do you exercise at least every alternate day ?',
       'Do you participate in some outdoor sports (Badminton, Tennis, Swimming, Running, etc) every week ?',
       'Do you participate in Pilates/ Yoga sessions every week ?',
       'Do you do the chores(cleaning, cooking or laundry) at home and if so, for how long ?'],
      dtype='object')

In [148]:
z = []
z.append(uact1['recommend'][17][0].keys())
z.append(uact1['recommend'][17][1].keys())

In [175]:
unique(uact1['recommend'][17])

['6fd4e033-3aae-4a27-9a1c-be3345be834b',
 '87d56a26-18c3-41db-aebd-27adbc05a8fc',
 '6c2d3398-0056-4c1b-9660-4f48752981f0',
 '186aff91-a875-49f8-8451-17be361b8178',
 'c964ab5d-950f-4057-b2c7-f7956b27690d',
 '39aac870-13d0-4d93-8285-c46ee2649520',
 '09e718a8-ad3e-4b3f-9868-c2eacc315632',
 '064aa5bc-3d41-41d9-ae38-eea0aeb7a90e',
 '28f88e8d-a4f7-423a-9716-8e2cc163cf8d',
 '9ae77239-924a-49c6-8dd8-1cc7128e8210',
 '581a5991-121a-444e-81d4-6bdd9c3cee7a',
 '78d31c7e-b2c2-4d55-8172-761233226bdd',
 'e9f5480c-145c-411c-8e99-7fce98070a11',
 '6dc81491-43c1-441f-9a0b-5da872c49aab',
 '6d62c06e-ec7b-40d4-8cc9-d2ac20411586',
 '5bc1b7c9-2021-4f90-b9a7-b2cf88faae66',
 'e746ecb0-2b6b-4cb6-9148-c59ee07630a3',
 '892d8c8c-e8a8-4be5-9cdc-de99f156cb57',
 'b2f3f52b-a703-49c0-9cdc-eaa399ccc81c']

In [185]:
unique(uact1['recommend'][103])

['6fd4e033-3aae-4a27-9a1c-be3345be834b',
 '87d56a26-18c3-41db-aebd-27adbc05a8fc',
 '6c2d3398-0056-4c1b-9660-4f48752981f0',
 '186aff91-a875-49f8-8451-17be361b8178',
 'c964ab5d-950f-4057-b2c7-f7956b27690d',
 '09e718a8-ad3e-4b3f-9868-c2eacc315632',
 '064aa5bc-3d41-41d9-ae38-eea0aeb7a90e',
 '581a5991-121a-444e-81d4-6bdd9c3cee7a',
 '78d31c7e-b2c2-4d55-8172-761233226bdd',
 '6d62c06e-ec7b-40d4-8cc9-d2ac20411586',
 '892d8c8c-e8a8-4be5-9cdc-de99f156cb57']

In [183]:
uact1.recommend

17      [09e718a8-ad3e-4b3f-9868-c2eacc315632, 6fd4e03...
48      [b2f3f52b-a703-49c0-9cdc-eaa399ccc81c, e746ecb...
76      [6c2d3398-0056-4c1b-9660-4f48752981f0, 892d8c8...
94                                                     []
103     [064aa5bc-3d41-41d9-ae38-eea0aeb7a90e, 78d31c7...
                              ...                        
1825    [09e718a8-ad3e-4b3f-9868-c2eacc315632, 6fd4e03...
1865    [78d31c7e-b2c2-4d55-8172-761233226bdd, 186aff9...
1905    [09e718a8-ad3e-4b3f-9868-c2eacc315632, 6fd4e03...
1938    [09e718a8-ad3e-4b3f-9868-c2eacc315632, 6fd4e03...
1940    [09e718a8-ad3e-4b3f-9868-c2eacc315632, 6fd4e03...
Name: recommend, Length: 87, dtype: object

In [151]:
uact1.recommend[17][1].keys()

Index(['78d31c7e-b2c2-4d55-8172-761233226bdd',
       '064aa5bc-3d41-41d9-ae38-eea0aeb7a90e',
       '87d56a26-18c3-41db-aebd-27adbc05a8fc'],
      dtype='object')

In [152]:
uact1['recommend'][17][1].keys()

Index(['78d31c7e-b2c2-4d55-8172-761233226bdd',
       '064aa5bc-3d41-41d9-ae38-eea0aeb7a90e',
       '87d56a26-18c3-41db-aebd-27adbc05a8fc'],
      dtype='object')

In [171]:
len(uact1['recommend'][48])

51

In [166]:
#adding all the recommendations in one list
for i in uact1.index:
    tp = list()
    for j in range(0,len(uact1['recommend'][i])):
        temp = list(uact1['recommend'][i][j].keys())
        tp = tp + temp
    uact1['recommend'][i]=tp


In [156]:
def unique(input_list):
	return list(set(input_list))
for i in uact1.index:
    uact1.recommend[i] = unique(uact1.recommend[i])

In [162]:
len(uact1['recommend'][94])

0